This notebook contains the entire ETL pipeline executed to generate the 2023 county data required for model extrapolation. The process is identical to how the model training/test data (2012, 2016 & 2019 - "County_Final_With_Outcomes_And_Segments") was prepared.

In [1]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

**Demographic Data Extraction, Transformation & Loading (2023)**
-
-----------

In [2]:
# Reading in 2023 U.S. Census Bureau demographic dataset
initial_pa_demographic_2023 = pd.read_excel("Resources/PA_Demographics_2023.xlsx")
initial_pa_demographic_2023.head(100)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,Label,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,...,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent
1,SEX AND AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total population,"106,748","1,224,825","64,074","165,631","432,821","120,273","645,984","198,413","130,668",...,"166,053","868,742","319,091","90,120","1,550,542","143,786","72,197","210,232","351,163","464,640"
3,Male,49.2%,48.8%,49.9%,49.5%,49.5%,49.7%,49.5%,49.8%,49.8%,...,51.0%,49.0%,49.6%,51.1%,47.3%,51.5%,51.8%,49.3%,49.4%,49.8%
4,Female,50.8%,51.2%,50.1%,50.5%,50.5%,50.3%,50.5%,50.2%,50.2%,...,49.0%,51.0%,50.4%,48.9%,52.7%,48.5%,48.2%,50.7%,50.6%,50.2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Total housing units,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
96,"CITIZEN, VOTING AGE POPULATION",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,"Citizen, 18 and over population","84,365","964,197","52,011","132,875","317,905","95,685","496,444","158,968","105,069",...,"129,834","648,145","248,741","71,155","1,128,301","112,585","58,668","167,243","286,130","354,959"
98,Male,49.1%,48.1%,49.7%,48.9%,48.7%,49.2%,49.2%,49.3%,49.2%,...,50.0%,48.3%,48.9%,51.1%,45.7%,51.8%,52.2%,49.0%,49.0%,49.3%


In [3]:
# Retrieving only rows that contain the desired county summary demographic statistics
# % Male, % Female, % Male > 18 Years Old, % Female > 18 Years Old, % White, % Black or African American, % American Indian and Alaska Native, % Asian
initial_pa_demographic_2023 = initial_pa_demographic_2023.iloc[[3, 4, 27, 28, 39, 40, 41, 49], :]
initial_pa_demographic_2023.head(8)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
3,Male,49.2%,48.8%,49.9%,49.5%,49.5%,49.7%,49.5%,49.8%,49.8%,...,51.0%,49.0%,49.6%,51.1%,47.3%,51.5%,51.8%,49.3%,49.4%,49.8%
4,Female,50.8%,51.2%,50.1%,50.5%,50.5%,50.3%,50.5%,50.2%,50.2%,...,49.0%,51.0%,50.4%,48.9%,52.7%,48.5%,48.2%,50.7%,50.6%,50.2%
27,Male,49.0%,48.2%,49.7%,48.7%,49.0%,49.2%,49.0%,49.4%,49.4%,...,49.9%,48.4%,49.1%,50.8%,46.3%,51.9%,51.9%,49.1%,49.0%,49.2%
28,Female,51.0%,51.8%,50.3%,51.3%,51.0%,50.8%,51.0%,50.6%,50.6%,...,50.1%,51.6%,50.9%,49.2%,53.7%,48.1%,48.1%,50.9%,51.0%,50.8%
39,White,88.2%,75.9%,95.1%,85.9%,69.2%,90.5%,80.9%,92.8%,90.6%,...,62.6%,72.3%,72.3%,89.1%,34.6%,86.2%,94.2%,90.3%,92.2%,80.2%
40,Black or African American,1.3%,12.5%,0.6%,5.4%,4.1%,2.4%,3.2%,0.9%,3.2%,...,14.4%,9.9%,5.4%,2.0%,39.5%,3.3%,2.2%,2.8%,2.7%,5.3%
41,American Indian and Alaska Native,0.4%,0.1%,0.0%,0.2%,0.7%,0.0%,0.2%,0.3%,0.0%,...,0.2%,0.2%,0.2%,0.0%,0.5%,0.2%,0.0%,0.4%,0.1%,0.1%
49,Asian,0.8%,4.2%,0.1%,0.3%,1.4%,0.8%,5.0%,1.4%,0.4%,...,2.7%,7.9%,3.3%,0.6%,8.0%,0.5%,0.7%,1.4%,0.9%,1.4%


In [4]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_demographic_2023 = initial_pa_demographic_2023.T
initial_pa_demographic_2023.head()


,3,4,27,28,39,40,41,49
Unnamed: 0,Male,Female,Male,Female,White,Black or African American,American Indian and Alaska Native,Asian
"Adams County, Pennsylvania",49.2%,50.8%,49.0%,51.0%,88.2%,1.3%,0.4%,0.8%
"Allegheny County, Pennsylvania",48.8%,51.2%,48.2%,51.8%,75.9%,12.5%,0.1%,4.2%
"Armstrong County, Pennsylvania",49.9%,50.1%,49.7%,50.3%,95.1%,0.6%,0.0%,0.1%
"Beaver County, Pennsylvania",49.5%,50.5%,48.7%,51.3%,85.9%,5.4%,0.2%,0.3%


In [5]:
# Resetting index and renaming to create a 'County' column
initial_pa_demographic_2023 = initial_pa_demographic_2023.reset_index()
initial_pa_demographic_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_demographic_2023.head()


,County,3,4,27,28,39,40,41,49
0,Unnamed: 0,Male,Female,Male,Female,White,Black or African American,American Indian and Alaska Native,Asian
1,"Adams County, Pennsylvania",49.2%,50.8%,49.0%,51.0%,88.2%,1.3%,0.4%,0.8%
2,"Allegheny County, Pennsylvania",48.8%,51.2%,48.2%,51.8%,75.9%,12.5%,0.1%,4.2%
3,"Armstrong County, Pennsylvania",49.9%,50.1%,49.7%,50.3%,95.1%,0.6%,0.0%,0.1%
4,"Beaver County, Pennsylvania",49.5%,50.5%,48.7%,51.3%,85.9%,5.4%,0.2%,0.3%


In [6]:
# Confirming column data types prior to renaming
print(initial_pa_demographic_2023.columns)


Index(['County', 3, 4, 27, 28, 39, 40, 41, 49], dtype='object')


In [7]:
# Converting column data types from integers to strings for renaming
initial_pa_demographic_2023.columns = initial_pa_demographic_2023.columns.astype(str)
print(initial_pa_demographic_2023.columns)


Index(['County', '3', '4', '27', '28', '39', '40', '41', '49'], dtype='object')


In [8]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_demographic_2023.replace('N', np.nan, inplace=True)


In [9]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_demographic_2023 = initial_pa_demographic_2023.rename(columns={'3': '% Male', '4': '% Female', '27': '% Male > 18 Years Old',
                                                                         '28': '% Female > 18 Years Old', '39': '% White', '40': '% Black or African American',
                                                                         '41': '% American Indian and Alaska Native', '49': '% Asian'})
initial_pa_demographic_2023 = initial_pa_demographic_2023.drop(0)
initial_pa_demographic_2023 = initial_pa_demographic_2023.reset_index(drop=True)
initial_pa_demographic_2023['County'] = initial_pa_demographic_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_demographic_2023['County'] = initial_pa_demographic_2023['County'].apply(lambda x: x.upper())
initial_pa_demographic_2023.insert(0, 'Year', 2023)
initial_pa_demographic_2023['% Male'] = initial_pa_demographic_2023['% Male'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% Female'] = initial_pa_demographic_2023['% Female'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% Male > 18 Years Old'] = initial_pa_demographic_2023['% Male > 18 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% Female > 18 Years Old'] = initial_pa_demographic_2023['% Female > 18 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% White'] = initial_pa_demographic_2023['% White'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% Black or African American'] = initial_pa_demographic_2023['% Black or African American'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% American Indian and Alaska Native'] = initial_pa_demographic_2023['% American Indian and Alaska Native'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023['% Asian'] = initial_pa_demographic_2023['% Asian'].str.rstrip('%').astype(float) / 100
initial_pa_demographic_2023


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007


**Education Data Extraction, Transformation & Loading (2023)**
-
-----------

In [10]:
# Reading in 2023 U.S. Census Bureau education dataset
initial_pa_education_2023 = pd.read_excel("Resources/PA_Education_2023.xlsx")
initial_pa_education_2023.head(35)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,...,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,AGE BY EDUCATIONAL ATTAINMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Population 18 to 24 years,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,Less than high school graduate,12.5%,8.7%,17.1%,8.9%,10.9%,9.9%,10.8%,8.6%,8.1%,...,8.1%,10.1%,5.8%,14.8%,10.1%,13.6%,5.8%,8.4%,12.1%,9.4%
5,High school graduate (includes equivalency),39.8%,37.4%,47.1%,43.1%,47.3%,38.7%,39.5%,48.7%,35.9%,...,41.5%,32.8%,25.7%,49.8%,36.8%,48.8%,65.9%,46.2%,41.4%,51.2%
6,Some college or associate's degree,40.6%,35.7%,25.1%,35.1%,31.5%,41.8%,31.8%,31.5%,45.0%,...,38.0%,32.8%,53.2%,31.3%,37.7%,27.4%,24.1%,34.7%,38.0%,29.4%
7,Bachelor's degree or higher,7.1%,18.2%,10.7%,12.9%,10.2%,9.6%,17.9%,11.2%,11.0%,...,12.4%,24.3%,15.4%,4.1%,15.4%,10.2%,4.2%,10.7%,8.5%,10.0%
8,Population 25 years and over,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
9,Less than 9th grade,3.6%,1.3%,1.4%,0.9%,4.4%,2.6%,1.5%,2.0%,2.8%,...,3.1%,1.8%,1.9%,2.0%,4.6%,2.6%,2.5%,1.9%,1.1%,2.8%


In [11]:
# Retrieving only rows that contain the desired county summary demographic statistics
# % High School Graduate & % Bachelor's degree or higher for 18 to 24 years old and % High School Graduate or Higher and % Bachelor's degree or higher for the remaining age groups:
# 25 to 34, 35 to 44, 45 to 64 and 65 & older
initial_pa_education_2023 = initial_pa_education_2023.iloc[[5, 7, 19, 20, 22, 23, 25, 26, 28, 29], :]
initial_pa_education_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
5,High school graduate (includes equivalency),39.8%,37.4%,47.1%,43.1%,47.3%,38.7%,39.5%,48.7%,35.9%,...,41.5%,32.8%,25.7%,49.8%,36.8%,48.8%,65.9%,46.2%,41.4%,51.2%
7,Bachelor's degree or higher,7.1%,18.2%,10.7%,12.9%,10.2%,9.6%,17.9%,11.2%,11.0%,...,12.4%,24.3%,15.4%,4.1%,15.4%,10.2%,4.2%,10.7%,8.5%,10.0%
19,High school graduate or higher,95.3%,98.3%,94.8%,96.9%,90.3%,94.2%,97.7%,91.8%,96.1%,...,92.8%,96.6%,98.8%,90.0%,94.3%,92.4%,93.9%,97.1%,97.0%,93.5%
20,Bachelor's degree or higher,24.6%,57.9%,25.3%,36.4%,25.9%,27.7%,50.1%,44.2%,28.2%,...,34.3%,56.3%,37.7%,24.8%,52.5%,20.8%,13.9%,42.3%,35.1%,32.8%
22,High school graduate or higher,93.5%,96.2%,94.5%,97.1%,88.9%,94.2%,97.0%,93.2%,97.3%,...,87.9%,94.9%,93.9%,91.4%,91.5%,90.9%,89.8%,96.5%,95.3%,93.3%
23,Bachelor's degree or higher,24.5%,57.8%,23.8%,40.0%,33.9%,23.9%,56.7%,47.2%,31.7%,...,29.2%,60.8%,42.1%,25.3%,42.2%,23.8%,23.0%,40.3%,43.3%,34.2%
25,High school graduate or higher,91.8%,96.7%,93.8%,97.4%,87.4%,91.4%,95.5%,96.0%,92.5%,...,91.1%,95.1%,94.3%,88.5%,85.1%,89.1%,90.1%,94.2%,96.6%,91.6%
26,Bachelor's degree or higher,24.2%,45.0%,18.7%,30.1%,27.9%,22.3%,43.6%,41.0%,22.9%,...,23.0%,52.9%,33.0%,17.9%,26.0%,20.8%,15.6%,30.4%,33.2%,27.9%
28,High school graduate or higher,88.7%,94.8%,93.2%,95.0%,85.7%,91.2%,94.1%,93.5%,89.4%,...,88.9%,94.1%,92.0%,89.9%,81.6%,88.8%,89.6%,93.1%,94.5%,89.0%
29,Bachelor's degree or higher,22.8%,34.5%,18.3%,22.1%,21.5%,18.7%,34.5%,24.5%,19.9%,...,26.8%,43.7%,29.5%,15.0%,23.9%,16.7%,14.8%,26.3%,26.5%,25.1%


In [12]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_education_2023 = initial_pa_education_2023.T
initial_pa_education_2023.head()


,5,7,19,20,22,23,25,26,28,29
Unnamed: 0,High school graduate (includes equivalency),Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher
"Adams County, Pennsylvania",39.8%,7.1%,95.3%,24.6%,93.5%,24.5%,91.8%,24.2%,88.7%,22.8%
"Allegheny County, Pennsylvania",37.4%,18.2%,98.3%,57.9%,96.2%,57.8%,96.7%,45.0%,94.8%,34.5%
"Armstrong County, Pennsylvania",47.1%,10.7%,94.8%,25.3%,94.5%,23.8%,93.8%,18.7%,93.2%,18.3%
"Beaver County, Pennsylvania",43.1%,12.9%,96.9%,36.4%,97.1%,40.0%,97.4%,30.1%,95.0%,22.1%


In [13]:
# Resetting index and renaming to create a 'County' column
initial_pa_education_2023 = initial_pa_education_2023.reset_index()
initial_pa_education_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_education_2023.head()


,County,5,7,19,20,22,23,25,26,28,29
0,Unnamed: 0,High school graduate (includes equivalency),Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher,High school graduate or higher,Bachelor's degree or higher
1,"Adams County, Pennsylvania",39.8%,7.1%,95.3%,24.6%,93.5%,24.5%,91.8%,24.2%,88.7%,22.8%
2,"Allegheny County, Pennsylvania",37.4%,18.2%,98.3%,57.9%,96.2%,57.8%,96.7%,45.0%,94.8%,34.5%
3,"Armstrong County, Pennsylvania",47.1%,10.7%,94.8%,25.3%,94.5%,23.8%,93.8%,18.7%,93.2%,18.3%
4,"Beaver County, Pennsylvania",43.1%,12.9%,96.9%,36.4%,97.1%,40.0%,97.4%,30.1%,95.0%,22.1%


In [14]:
# Confirming column data types prior to renaming
print(initial_pa_education_2023.columns)


Index(['County', 5, 7, 19, 20, 22, 23, 25, 26, 28, 29], dtype='object')


In [15]:
# Converting column data types from integers to strings for renaming
initial_pa_education_2023.columns = initial_pa_education_2023.columns.astype(str)
print(initial_pa_education_2023.columns)


Index(['County', '5', '7', '19', '20', '22', '23', '25', '26', '28', '29'], dtype='object')


In [16]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_education_2023.replace('N', np.nan, inplace=True)


In [17]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_education_2023 = initial_pa_education_2023.rename(columns={'5': '% High School Graduate (18-24)', '7': '% Bachelors Degree or Higher (18-24)', '19': '% High School Graduate or Higher (25-34)',
                                                                         '20': '% Bachelors Degree or Higher (25-34)', '22': '% High School Graduate or Higher (35-44)', '23': '% Bachelors Degree or Higher (35-44)',
                                                                         '25': '% High School Graduate or Higher (45-64)', '26': '% Bachelors Degree or Higher (45-64)',
                                                                         '28': '% High School Graduate or Higher (65 & Older)', '29': '% Bachelors Degree or Higher (65 & Older)'})
initial_pa_education_2023 = initial_pa_education_2023.drop(0)
initial_pa_education_2023 = initial_pa_education_2023.reset_index(drop=True)
initial_pa_education_2023['County'] = initial_pa_education_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_education_2023['County'] = initial_pa_education_2023['County'].apply(lambda x: x.upper())
initial_pa_education_2023.insert(0, 'Year', 2023)
initial_pa_education_2023['% High School Graduate (18-24)'] = initial_pa_education_2023['% High School Graduate (18-24)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% Bachelors Degree or Higher (18-24)'] = initial_pa_education_2023['% Bachelors Degree or Higher (18-24)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% High School Graduate or Higher (25-34)'] = initial_pa_education_2023['% High School Graduate or Higher (25-34)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% Bachelors Degree or Higher (25-34)'] = initial_pa_education_2023['% Bachelors Degree or Higher (25-34)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% High School Graduate or Higher (35-44)'] = initial_pa_education_2023['% High School Graduate or Higher (35-44)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% Bachelors Degree or Higher (35-44)'] = initial_pa_education_2023['% Bachelors Degree or Higher (35-44)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% High School Graduate or Higher (45-64)'] = initial_pa_education_2023['% High School Graduate or Higher (45-64)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% Bachelors Degree or Higher (45-64)'] = initial_pa_education_2023['% Bachelors Degree or Higher (45-64)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% High School Graduate or Higher (65 & Older)'] = initial_pa_education_2023['% High School Graduate or Higher (65 & Older)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023['% Bachelors Degree or Higher (65 & Older)'] = initial_pa_education_2023['% Bachelors Degree or Higher (65 & Older)'].str.rstrip('%').astype(float) / 100
initial_pa_education_2023


,Year,County,% High School Graduate (18-24),% Bachelors Degree or Higher (18-24),% High School Graduate or Higher (25-34),% Bachelors Degree or Higher (25-34),% High School Graduate or Higher (35-44),% Bachelors Degree or Higher (35-44),% High School Graduate or Higher (45-64),% Bachelors Degree or Higher (45-64),% High School Graduate or Higher (65 & Older),% Bachelors Degree or Higher (65 & Older)
0,2023,ADAMS,0.398,0.071,0.953,0.246,0.935,0.245,0.918,0.242,0.887,0.228
1,2023,ALLEGHENY,0.374,0.182,0.983,0.579,0.962,0.578,0.967,0.450,0.948,0.345
2,2023,ARMSTRONG,0.471,0.107,0.948,0.253,0.945,0.238,0.938,0.187,0.932,0.183
3,2023,BEAVER,0.431,0.129,0.969,0.364,0.971,0.400,0.974,0.301,0.950,0.221
4,2023,BERKS,0.473,0.102,0.903,0.259,0.889,0.339,0.874,0.279,0.857,0.215
5,2023,BLAIR,0.387,0.096,0.942,0.277,0.942,0.239,0.914,0.223,0.912,0.187
6,2023,BUCKS,0.395,0.179,0.977,0.501,0.970,0.567,0.955,0.436,0.941,0.345
7,2023,BUTLER,0.487,0.112,0.918,0.442,0.932,0.472,0.960,0.410,0.935,0.245
8,2023,CAMBRIA,0.359,0.110,0.961,0.282,0.973,0.317,0.925,0.229,0.894,0.199
9,2023,CARBON,0.541,0.079,0.951,0.239,0.925,0.232,0.897,0.176,0.908,0.140


**Income (Households) Data Extraction, Transformation & Loading (2023)**
-
-----------

In [18]:
# Reading in 2023 U.S. Census Bureau income (household) dataset
initial_pa_households_2023 = pd.read_excel("Resources/PA_Income_Households_2023.xlsx")
initial_pa_households_2023.head(19)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Households,Households,Households,Households,Households,Households,Households,Households,Households,...,Households,Households,Households,Households,Households,Households,Households,Households,Households,Households
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Total,"41,810","547,396","28,117","73,220","163,307","50,651","249,189","81,535","57,282",...,"61,129","338,467","123,536","37,343","688,112","58,809","29,435","89,091","155,459","183,864"
3,"Less than $10,000",2.2%,6.2%,5.2%,3.8%,5.4%,6.1%,2.9%,4.7%,5.1%,...,3.4%,3.7%,4.2%,5.2%,9.3%,6.0%,4.3%,3.4%,4.9%,3.8%
4,"$10,000 to $14,999",2.7%,3.5%,4.1%,4.1%,2.4%,5.7%,2.0%,2.0%,4.6%,...,2.5%,2.1%,2.5%,5.2%,6.0%,5.9%,3.2%,3.7%,3.4%,2.6%
5,"$15,000 to $24,999",5.7%,6.3%,8.7%,7.3%,7.4%,11.1%,4.7%,7.3%,11.0%,...,6.6%,4.3%,4.2%,8.2%,8.6%,9.2%,12.7%,6.5%,7.4%,4.5%
6,"$25,000 to $34,999",7.7%,6.8%,8.4%,7.8%,6.5%,8.8%,3.9%,5.7%,11.2%,...,10.0%,5.3%,6.3%,8.2%,7.6%,8.1%,10.3%,7.1%,7.7%,6.5%
7,"$35,000 to $49,999",9.6%,10.4%,13.3%,12.7%,10.5%,13.7%,6.6%,10.5%,12.5%,...,13.2%,7.2%,9.5%,13.8%,11.1%,10.9%,12.7%,10.7%,11.5%,11.2%
8,"$50,000 to $74,999",20.7%,16.0%,19.9%,20.2%,17.4%,19.0%,14.7%,17.2%,19.1%,...,15.9%,12.4%,18.1%,21.8%,17.0%,17.2%,20.0%,17.3%,17.7%,17.4%
9,"$75,000 to $99,999",15.4%,12.5%,13.9%,12.4%,13.0%,11.9%,11.3%,11.7%,11.6%,...,11.4%,11.5%,13.4%,13.8%,12.1%,16.0%,15.3%,12.3%,13.1%,16.0%


In [19]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Four groupings = Households, Families, Married-couple families & non family households
# % of groups above that fall into the income categories: Less Than $10,000, $10,000-$14,999, $15,000-$24,999, $25,000-$34,999,
# $35,000-$49,999, $50,000-$74,999, $75,000-$99,999, $100,000-$149,999, $150,000-$199,999, $200,000 or More
# Total count for each grouping
# Median income for each grouping
# Mean income for each grouping
initial_pa_households_2023 = initial_pa_households_2023.iloc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], :]
initial_pa_households_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
2,Total,"41,810","547,396","28,117","73,220","163,307","50,651","249,189","81,535","57,282",...,"61,129","338,467","123,536","37,343","688,112","58,809","29,435","89,091","155,459","183,864"
3,"Less than $10,000",2.2%,6.2%,5.2%,3.8%,5.4%,6.1%,2.9%,4.7%,5.1%,...,3.4%,3.7%,4.2%,5.2%,9.3%,6.0%,4.3%,3.4%,4.9%,3.8%
4,"$10,000 to $14,999",2.7%,3.5%,4.1%,4.1%,2.4%,5.7%,2.0%,2.0%,4.6%,...,2.5%,2.1%,2.5%,5.2%,6.0%,5.9%,3.2%,3.7%,3.4%,2.6%
5,"$15,000 to $24,999",5.7%,6.3%,8.7%,7.3%,7.4%,11.1%,4.7%,7.3%,11.0%,...,6.6%,4.3%,4.2%,8.2%,8.6%,9.2%,12.7%,6.5%,7.4%,4.5%
6,"$25,000 to $34,999",7.7%,6.8%,8.4%,7.8%,6.5%,8.8%,3.9%,5.7%,11.2%,...,10.0%,5.3%,6.3%,8.2%,7.6%,8.1%,10.3%,7.1%,7.7%,6.5%
7,"$35,000 to $49,999",9.6%,10.4%,13.3%,12.7%,10.5%,13.7%,6.6%,10.5%,12.5%,...,13.2%,7.2%,9.5%,13.8%,11.1%,10.9%,12.7%,10.7%,11.5%,11.2%
8,"$50,000 to $74,999",20.7%,16.0%,19.9%,20.2%,17.4%,19.0%,14.7%,17.2%,19.1%,...,15.9%,12.4%,18.1%,21.8%,17.0%,17.2%,20.0%,17.3%,17.7%,17.4%
9,"$75,000 to $99,999",15.4%,12.5%,13.9%,12.4%,13.0%,11.9%,11.3%,11.7%,11.6%,...,11.4%,11.5%,13.4%,13.8%,12.1%,16.0%,15.3%,12.3%,13.1%,16.0%
10,"$100,000 to $149,999",18.0%,16.9%,16.4%,17.0%,19.5%,14.6%,19.5%,17.2%,13.8%,...,19.4%,19.4%,20.8%,14.2%,13.7%,17.2%,14.1%,20.2%,17.5%,19.8%
11,"$150,000 to $199,999",8.8%,9.1%,5.5%,7.6%,9.6%,5.2%,14.5%,10.4%,5.6%,...,8.9%,11.9%,9.2%,6.5%,6.8%,5.6%,5.4%,8.8%,8.8%,9.1%


In [20]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_households_2023 = initial_pa_households_2023.T
initial_pa_households_2023.head()


,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
"Adams County, Pennsylvania","41,810",2.2%,2.7%,5.7%,7.7%,9.6%,20.7%,15.4%,18.0%,8.8%,9.2%,"76,725","97,534"
"Allegheny County, Pennsylvania","547,396",6.2%,3.5%,6.3%,6.8%,10.4%,16.0%,12.5%,16.9%,9.1%,12.3%,"76,615","108,244"
"Armstrong County, Pennsylvania","28,117",5.2%,4.1%,8.7%,8.4%,13.3%,19.9%,13.9%,16.4%,5.5%,4.6%,"61,836","80,274"
"Beaver County, Pennsylvania","73,220",3.8%,4.1%,7.3%,7.8%,12.7%,20.2%,12.4%,17.0%,7.6%,7.1%,"66,768","86,549"


In [21]:
# Resetting index and renaming to create a 'County' column
initial_pa_households_2023 = initial_pa_households_2023.reset_index()
initial_pa_households_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_households_2023.head()


,County,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
1,"Adams County, Pennsylvania","41,810",2.2%,2.7%,5.7%,7.7%,9.6%,20.7%,15.4%,18.0%,8.8%,9.2%,"76,725","97,534"
2,"Allegheny County, Pennsylvania","547,396",6.2%,3.5%,6.3%,6.8%,10.4%,16.0%,12.5%,16.9%,9.1%,12.3%,"76,615","108,244"
3,"Armstrong County, Pennsylvania","28,117",5.2%,4.1%,8.7%,8.4%,13.3%,19.9%,13.9%,16.4%,5.5%,4.6%,"61,836","80,274"
4,"Beaver County, Pennsylvania","73,220",3.8%,4.1%,7.3%,7.8%,12.7%,20.2%,12.4%,17.0%,7.6%,7.1%,"66,768","86,549"


In [22]:
# Confirming column data types prior to renaming
print(initial_pa_households_2023.columns)


Index(['County', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='object')


In [23]:
# Converting column data types from integers to strings for renaming
initial_pa_households_2023.columns = initial_pa_households_2023.columns.astype(str)
print(initial_pa_households_2023.columns)


Index(['County', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'],
      dtype='object')


In [24]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_households_2023.replace('N', np.nan, inplace=True)


In [25]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_households_2023 = initial_pa_households_2023.rename(columns={'2': '# of Households', '3': '% Households <$10,000', '4': '% Households $10,000-$14,999',
                                                                         '5': '% Households $15,000-$24,999', '6': '% Households $25,000-$34,999', '7': '% Households $35,000-$49,999',
                                                                         '8': '% Households $50,000-$74,999', '9': '% Households $75,000-$99,999',
                                                                         '10': '% Households $100,000-$149,999', '11': '% Households $150,000-$199,999',
                                                                        '12': '% Households $200,000 or More', '13': 'Median Household Income ($)',
                                                                       '14': 'Mean Household Income ($)'})
initial_pa_households_2023 = initial_pa_households_2023.drop(0)
initial_pa_households_2023 = initial_pa_households_2023.reset_index(drop=True)
initial_pa_households_2023['County'] = initial_pa_households_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_households_2023['County'] = initial_pa_households_2023['County'].apply(lambda x: x.upper())
initial_pa_households_2023.insert(0, 'Year', 2023)
initial_pa_households_2023['% Households <$10,000'] = initial_pa_households_2023['% Households <$10,000'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $10,000-$14,999'] = initial_pa_households_2023['% Households $10,000-$14,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $15,000-$24,999'] = initial_pa_households_2023['% Households $15,000-$24,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $25,000-$34,999'] = initial_pa_households_2023['% Households $25,000-$34,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $35,000-$49,999'] = initial_pa_households_2023['% Households $35,000-$49,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $50,000-$74,999'] = initial_pa_households_2023['% Households $50,000-$74,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $75,000-$99,999'] = initial_pa_households_2023['% Households $75,000-$99,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $100,000-$149,999'] = initial_pa_households_2023['% Households $100,000-$149,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $150,000-$199,999'] = initial_pa_households_2023['% Households $150,000-$199,999'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['% Households $200,000 or More'] = initial_pa_households_2023['% Households $200,000 or More'].str.rstrip('%').astype(float) / 100
initial_pa_households_2023['# of Households'] = initial_pa_households_2023['# of Households'].str.replace(',', '').astype(float)
initial_pa_households_2023['Median Household Income ($)'] = initial_pa_households_2023['Median Household Income ($)'].str.replace(',', '').astype(float)
initial_pa_households_2023['Mean Household Income ($)'] = initial_pa_households_2023['Mean Household Income ($)'].str.replace(',', '').astype(float)
initial_pa_households_2023


,Year,County,# of Households,"% Households <$10,000","% Households $10,000-$14,999","% Households $15,000-$24,999","% Households $25,000-$34,999","% Households $35,000-$49,999","% Households $50,000-$74,999","% Households $75,000-$99,999","% Households $100,000-$149,999","% Households $150,000-$199,999","% Households $200,000 or More",Median Household Income ($),Mean Household Income ($)
0,2023,ADAMS,41810.0,0.022,0.027,0.057,0.077,0.096,0.207,0.154,0.180,0.088,0.092,76725.0,97534.0
1,2023,ALLEGHENY,547396.0,0.062,0.035,0.063,0.068,0.104,0.160,0.125,0.169,0.091,0.123,76615.0,108244.0
2,2023,ARMSTRONG,28117.0,0.052,0.041,0.087,0.084,0.133,0.199,0.139,0.164,0.055,0.046,61836.0,80274.0
3,2023,BEAVER,73220.0,0.038,0.041,0.073,0.078,0.127,0.202,0.124,0.170,0.076,0.071,66768.0,86549.0
4,2023,BERKS,163307.0,0.054,0.024,0.074,0.065,0.105,0.174,0.130,0.195,0.096,0.083,75675.0,97411.0
5,2023,BLAIR,50651.0,0.061,0.057,0.111,0.088,0.137,0.190,0.119,0.146,0.052,0.038,54002.0,72354.0
6,2023,BUCKS,249189.0,0.029,0.020,0.047,0.039,0.066,0.147,0.113,0.195,0.145,0.199,107221.0,140231.0
7,2023,BUTLER,81535.0,0.047,0.020,0.073,0.057,0.105,0.172,0.117,0.172,0.104,0.133,80379.0,110349.0
8,2023,CAMBRIA,57282.0,0.051,0.046,0.110,0.112,0.125,0.191,0.116,0.138,0.056,0.054,55748.0,79887.0
9,2023,CARBON,28934.0,0.046,0.020,0.093,0.099,0.141,0.166,0.138,0.185,0.060,0.051,64229.0,82247.0


**Income (Families) Data Extraction, Transformation & Loading (2023)**
-
-----------

In [26]:
# Reading in 2023 U.S. Census Bureau income (families) dataset
initial_pa_families_2023 = pd.read_excel("Resources/PA_Income_Families_2023.xlsx")
initial_pa_families_2023.head(19)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Families,Families,Families,Families,Families,Families,Families,Families,Families,...,Families,Families,Families,Families,Families,Families,Families,Families,Families,Families
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Total,"28,604","306,132","18,962","44,632","111,457","29,519","178,826","51,625","34,225",...,"42,605","221,197","83,182","23,395","352,837","36,033","18,564","57,214","96,790","122,781"
3,"Less than $10,000",2.3%,3.6%,1.8%,2.4%,3.2%,2.8%,2.0%,2.1%,2.2%,...,1.9%,2.3%,2.2%,3.7%,5.9%,3.0%,1.7%,1.5%,2.6%,2.4%
4,"$10,000 to $14,999",2.5%,1.7%,1.6%,2.0%,1.6%,2.0%,1.0%,0.6%,1.3%,...,2.2%,0.7%,2.0%,2.9%,3.4%,3.4%,0.7%,1.8%,1.4%,1.5%
5,"$15,000 to $24,999",2.5%,2.9%,4.6%,4.5%,4.6%,5.2%,2.8%,4.1%,4.5%,...,4.0%,2.1%,2.1%,3.1%,7.5%,3.9%,5.8%,3.2%,3.9%,2.3%
6,"$25,000 to $34,999",4.5%,4.0%,6.1%,6.4%,4.6%,4.5%,2.0%,3.5%,7.8%,...,8.5%,3.3%,5.8%,5.8%,7.3%,5.2%,6.9%,4.4%,4.7%,3.9%
7,"$35,000 to $49,999",7.2%,7.8%,12.9%,9.4%,10.0%,13.8%,4.2%,7.3%,11.9%,...,11.4%,4.7%,5.4%,13.9%,10.1%,11.6%,12.3%,7.8%,8.5%,8.7%
8,"$50,000 to $74,999",18.4%,14.0%,21.9%,18.7%,15.6%,19.9%,12.3%,15.3%,19.5%,...,15.3%,9.6%,17.0%,20.5%,17.4%,19.9%,22.2%,16.5%,17.8%,15.1%
9,"$75,000 to $99,999",16.3%,13.5%,17.4%,12.8%,14.4%,17.7%,11.8%,12.4%,16.5%,...,11.7%,10.9%,14.5%,18.1%,13.8%,18.6%,19.5%,14.3%,15.4%,18.0%


In [27]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Four groupings = Households, Families, Married-couple families & non family households
# % of groups above that fall into the income categories: Less Than $10,000, $10,000-$14,999, $15,000-$24,999, $25,000-$34,999,
# $35,000-$49,999, $50,000-$74,999, $75,000-$99,999, $100,000-$149,999, $150,000-$199,999, $200,000 or More
# Total count for each grouping
# Median income for each grouping
# Mean income for each grouping
initial_pa_families_2023 = initial_pa_families_2023.iloc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], :]
initial_pa_families_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
2,Total,"28,604","306,132","18,962","44,632","111,457","29,519","178,826","51,625","34,225",...,"42,605","221,197","83,182","23,395","352,837","36,033","18,564","57,214","96,790","122,781"
3,"Less than $10,000",2.3%,3.6%,1.8%,2.4%,3.2%,2.8%,2.0%,2.1%,2.2%,...,1.9%,2.3%,2.2%,3.7%,5.9%,3.0%,1.7%,1.5%,2.6%,2.4%
4,"$10,000 to $14,999",2.5%,1.7%,1.6%,2.0%,1.6%,2.0%,1.0%,0.6%,1.3%,...,2.2%,0.7%,2.0%,2.9%,3.4%,3.4%,0.7%,1.8%,1.4%,1.5%
5,"$15,000 to $24,999",2.5%,2.9%,4.6%,4.5%,4.6%,5.2%,2.8%,4.1%,4.5%,...,4.0%,2.1%,2.1%,3.1%,7.5%,3.9%,5.8%,3.2%,3.9%,2.3%
6,"$25,000 to $34,999",4.5%,4.0%,6.1%,6.4%,4.6%,4.5%,2.0%,3.5%,7.8%,...,8.5%,3.3%,5.8%,5.8%,7.3%,5.2%,6.9%,4.4%,4.7%,3.9%
7,"$35,000 to $49,999",7.2%,7.8%,12.9%,9.4%,10.0%,13.8%,4.2%,7.3%,11.9%,...,11.4%,4.7%,5.4%,13.9%,10.1%,11.6%,12.3%,7.8%,8.5%,8.7%
8,"$50,000 to $74,999",18.4%,14.0%,21.9%,18.7%,15.6%,19.9%,12.3%,15.3%,19.5%,...,15.3%,9.6%,17.0%,20.5%,17.4%,19.9%,22.2%,16.5%,17.8%,15.1%
9,"$75,000 to $99,999",16.3%,13.5%,17.4%,12.8%,14.4%,17.7%,11.8%,12.4%,16.5%,...,11.7%,10.9%,14.5%,18.1%,13.8%,18.6%,19.5%,14.3%,15.4%,18.0%
10,"$100,000 to $149,999",21.8%,21.2%,19.9%,22.2%,22.5%,20.5%,21.1%,21.4%,18.7%,...,22.6%,21.5%,23.8%,17.9%,15.9%,20.4%,20.3%,26.2%,21.6%,24.1%
11,"$150,000 to $199,999",11.7%,12.8%,7.4%,11.1%,12.7%,8.0%,17.9%,14.0%,8.9%,...,10.6%,14.7%,11.5%,9.3%,8.0%,7.8%,7.9%,11.1%,12.5%,11.8%


In [28]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_families_2023 = initial_pa_families_2023.T
initial_pa_families_2023.head()


,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
"Adams County, Pennsylvania","28,604",2.3%,2.5%,2.5%,4.5%,7.2%,18.4%,16.3%,21.8%,11.7%,12.8%,"94,782","114,160"
"Allegheny County, Pennsylvania","306,132",3.6%,1.7%,2.9%,4.0%,7.8%,14.0%,13.5%,21.2%,12.8%,18.5%,"104,863","139,153"
"Armstrong County, Pennsylvania","18,962",1.8%,1.6%,4.6%,6.1%,12.9%,21.9%,17.4%,19.9%,7.4%,6.3%,"76,316","95,141"
"Beaver County, Pennsylvania","44,632",2.4%,2.0%,4.5%,6.4%,9.4%,18.7%,12.8%,22.2%,11.1%,10.6%,"87,260","105,898"


In [29]:
# Resetting index and renaming to create a 'County' column
initial_pa_families_2023 = initial_pa_families_2023.reset_index()
initial_pa_families_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_families_2023.head()


,County,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
1,"Adams County, Pennsylvania","28,604",2.3%,2.5%,2.5%,4.5%,7.2%,18.4%,16.3%,21.8%,11.7%,12.8%,"94,782","114,160"
2,"Allegheny County, Pennsylvania","306,132",3.6%,1.7%,2.9%,4.0%,7.8%,14.0%,13.5%,21.2%,12.8%,18.5%,"104,863","139,153"
3,"Armstrong County, Pennsylvania","18,962",1.8%,1.6%,4.6%,6.1%,12.9%,21.9%,17.4%,19.9%,7.4%,6.3%,"76,316","95,141"
4,"Beaver County, Pennsylvania","44,632",2.4%,2.0%,4.5%,6.4%,9.4%,18.7%,12.8%,22.2%,11.1%,10.6%,"87,260","105,898"


In [30]:
# Confirming column data types prior to renaming
print(initial_pa_families_2023.columns)


Index(['County', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='object')


In [31]:
# Converting column data types from integers to strings for renaming
initial_pa_families_2023.columns = initial_pa_families_2023.columns.astype(str)
print(initial_pa_families_2023.columns)


Index(['County', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'],
      dtype='object')


In [32]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_families_2023.replace('N', np.nan, inplace=True)


In [33]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_families_2023 = initial_pa_families_2023.rename(columns={'2': '# of Families', '3': '% Families <$10,000', '4': '% Families $10,000-$14,999',
                                                                         '5': '% Families $15,000-$24,999', '6': '% Families $25,000-$34,999', '7': '% Families $35,000-$49,999',
                                                                         '8': '% Families $50,000-$74,999', '9': '% Families $75,000-$99,999',
                                                                         '10': '% Families $100,000-$149,999', '11': '% Families $150,000-$199,999',
                                                                        '12': '% Families $200,000 or More', '13': 'Median Families Income ($)',
                                                                       '14': 'Mean Families Income ($)'})
initial_pa_families_2023 = initial_pa_families_2023.drop(0)
initial_pa_families_2023 = initial_pa_families_2023.reset_index(drop=True)
initial_pa_families_2023['County'] = initial_pa_families_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_families_2023['County'] = initial_pa_families_2023['County'].apply(lambda x: x.upper())
initial_pa_families_2023.insert(0, 'Year', 2023)
initial_pa_families_2023['% Families <$10,000'] = initial_pa_families_2023['% Families <$10,000'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $10,000-$14,999'] = initial_pa_families_2023['% Families $10,000-$14,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $15,000-$24,999'] = initial_pa_families_2023['% Families $15,000-$24,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $25,000-$34,999'] = initial_pa_families_2023['% Families $25,000-$34,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $35,000-$49,999'] = initial_pa_families_2023['% Families $35,000-$49,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $50,000-$74,999'] = initial_pa_families_2023['% Families $50,000-$74,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $75,000-$99,999'] = initial_pa_families_2023['% Families $75,000-$99,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $100,000-$149,999'] = initial_pa_families_2023['% Families $100,000-$149,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $150,000-$199,999'] = initial_pa_families_2023['% Families $150,000-$199,999'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['% Families $200,000 or More'] = initial_pa_families_2023['% Families $200,000 or More'].str.rstrip('%').astype(float) / 100
initial_pa_families_2023['# of Families'] = initial_pa_families_2023['# of Families'].str.replace(',', '').astype(float)
initial_pa_families_2023['Median Families Income ($)'] = initial_pa_families_2023['Median Families Income ($)'].str.replace(',', '').astype(float)
initial_pa_families_2023['Mean Families Income ($)'] = initial_pa_families_2023['Mean Families Income ($)'].str.replace(',', '').astype(float)
initial_pa_families_2023


,Year,County,# of Families,"% Families <$10,000","% Families $10,000-$14,999","% Families $15,000-$24,999","% Families $25,000-$34,999","% Families $35,000-$49,999","% Families $50,000-$74,999","% Families $75,000-$99,999","% Families $100,000-$149,999","% Families $150,000-$199,999","% Families $200,000 or More",Median Families Income ($),Mean Families Income ($)
0,2023,ADAMS,28604.0,0.023,0.025,0.025,0.045,0.072,0.184,0.163,0.218,0.117,0.128,94782.0,114160.0
1,2023,ALLEGHENY,306132.0,0.036,0.017,0.029,0.040,0.078,0.140,0.135,0.212,0.128,0.185,104863.0,139153.0
2,2023,ARMSTRONG,18962.0,0.018,0.016,0.046,0.061,0.129,0.219,0.174,0.199,0.074,0.063,76316.0,95141.0
3,2023,BEAVER,44632.0,0.024,0.020,0.045,0.064,0.094,0.187,0.128,0.222,0.111,0.106,87260.0,105898.0
4,2023,BERKS,111457.0,0.032,0.016,0.046,0.046,0.100,0.156,0.144,0.225,0.127,0.107,92374.0,113536.0
5,2023,BLAIR,29519.0,0.028,0.020,0.052,0.045,0.138,0.199,0.177,0.205,0.080,0.056,77520.0,92021.0
6,2023,BUCKS,178826.0,0.020,0.010,0.028,0.020,0.042,0.123,0.118,0.211,0.179,0.249,131258.0,163288.0
7,2023,BUTLER,51625.0,0.021,0.006,0.041,0.035,0.073,0.153,0.124,0.214,0.140,0.193,108733.0,139060.0
8,2023,CAMBRIA,34225.0,0.022,0.013,0.045,0.078,0.119,0.195,0.165,0.187,0.089,0.087,78680.0,105586.0
9,2023,CARBON,18255.0,0.022,0.005,0.076,0.052,0.153,0.173,0.180,0.203,0.056,0.079,77005.0,94383.0


**Income (Married Couple Families) Data Extraction, Transformation & Loading (2023)**
-
-----------

In [34]:
# Reading in 2023 U.S. Census Bureau income (Married Couple Families) dataset
initial_pa_mc_families_2023 = pd.read_excel("Resources/PA_Income_Married_Couple_Families_2023.xlsx")
initial_pa_mc_families_2023.head(19)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,...,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families,Married-couple families
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Total,N,"229,325",N,"34,102","80,739","21,790","145,371","42,556","25,295",...,"30,968","177,397","63,745",N,"195,989","24,704",N,"44,874","75,549","95,483"
3,"Less than $10,000",N,1.0%,N,1.1%,1.6%,0.7%,0.9%,0.7%,1.2%,...,2.0%,1.1%,0.3%,N,2.5%,0.6%,N,0.5%,1.2%,0.8%
4,"$10,000 to $14,999",N,0.8%,N,1.2%,0.9%,1.9%,0.5%,0.0%,0.4%,...,2.6%,0.6%,0.6%,N,1.6%,0.3%,N,0.5%,0.7%,0.6%
5,"$15,000 to $24,999",N,1.5%,N,2.7%,2.3%,4.0%,2.1%,3.2%,2.9%,...,3.0%,1.1%,1.7%,N,4.1%,1.9%,N,1.7%,2.8%,1.4%
6,"$25,000 to $34,999",N,2.8%,N,3.9%,2.4%,2.6%,1.3%,2.5%,5.2%,...,4.7%,2.3%,3.8%,N,4.5%,4.2%,N,3.4%,2.2%,1.7%
7,"$35,000 to $49,999",N,5.5%,N,7.0%,8.5%,10.4%,3.9%,5.7%,10.3%,...,6.1%,3.8%,3.6%,N,6.8%,8.3%,N,5.6%,7.5%,6.7%
8,"$50,000 to $74,999",N,11.9%,N,17.9%,13.3%,19.2%,10.6%,13.5%,18.0%,...,13.8%,8.2%,15.0%,N,15.8%,17.9%,N,14.0%,16.4%,13.7%
9,"$75,000 to $99,999",N,13.7%,N,13.5%,14.9%,18.4%,10.5%,11.6%,16.0%,...,11.3%,9.6%,15.7%,N,14.7%,21.8%,N,14.6%,15.5%,18.7%


In [35]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Four groupings = Households, Families, Married-couple families & non family households
# % of groups above that fall into the income categories: Less Than $10,000, $10,000-$14,999, $15,000-$24,999, $25,000-$34,999,
# $35,000-$49,999, $50,000-$74,999, $75,000-$99,999, $100,000-$149,999, $150,000-$199,999, $200,000 or More
# Total count for each grouping
# Median income for each grouping
# Mean income for each grouping
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.iloc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], :]
initial_pa_mc_families_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
2,Total,N,"229,325",N,"34,102","80,739","21,790","145,371","42,556","25,295",...,"30,968","177,397","63,745",N,"195,989","24,704",N,"44,874","75,549","95,483"
3,"Less than $10,000",N,1.0%,N,1.1%,1.6%,0.7%,0.9%,0.7%,1.2%,...,2.0%,1.1%,0.3%,N,2.5%,0.6%,N,0.5%,1.2%,0.8%
4,"$10,000 to $14,999",N,0.8%,N,1.2%,0.9%,1.9%,0.5%,0.0%,0.4%,...,2.6%,0.6%,0.6%,N,1.6%,0.3%,N,0.5%,0.7%,0.6%
5,"$15,000 to $24,999",N,1.5%,N,2.7%,2.3%,4.0%,2.1%,3.2%,2.9%,...,3.0%,1.1%,1.7%,N,4.1%,1.9%,N,1.7%,2.8%,1.4%
6,"$25,000 to $34,999",N,2.8%,N,3.9%,2.4%,2.6%,1.3%,2.5%,5.2%,...,4.7%,2.3%,3.8%,N,4.5%,4.2%,N,3.4%,2.2%,1.7%
7,"$35,000 to $49,999",N,5.5%,N,7.0%,8.5%,10.4%,3.9%,5.7%,10.3%,...,6.1%,3.8%,3.6%,N,6.8%,8.3%,N,5.6%,7.5%,6.7%
8,"$50,000 to $74,999",N,11.9%,N,17.9%,13.3%,19.2%,10.6%,13.5%,18.0%,...,13.8%,8.2%,15.0%,N,15.8%,17.9%,N,14.0%,16.4%,13.7%
9,"$75,000 to $99,999",N,13.7%,N,13.5%,14.9%,18.4%,10.5%,11.6%,16.0%,...,11.3%,9.6%,15.7%,N,14.7%,21.8%,N,14.6%,15.5%,18.7%
10,"$100,000 to $149,999",N,23.8%,N,26.2%,26.0%,25.2%,21.6%,24.0%,23.2%,...,27.3%,21.0%,25.8%,N,21.2%,26.2%,N,29.2%,25.1%,27.5%
11,"$150,000 to $199,999",N,15.3%,N,13.0%,16.5%,10.5%,20.2%,16.3%,11.3%,...,13.4%,16.5%,14.0%,N,11.1%,10.3%,N,13.7%,14.3%,14.2%


In [36]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.T
initial_pa_mc_families_2023.head()


,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
"Adams County, Pennsylvania",N,N,N,N,N,N,N,N,N,N,N,"110,730",N
"Allegheny County, Pennsylvania","229,325",1.0%,0.8%,1.5%,2.8%,5.5%,11.9%,13.7%,23.8%,15.3%,23.6%,"124,087","162,677"
"Armstrong County, Pennsylvania",N,N,N,N,N,N,N,N,N,N,N,"87,175",N
"Beaver County, Pennsylvania","34,102",1.1%,1.2%,2.7%,3.9%,7.0%,17.9%,13.5%,26.2%,13.0%,13.5%,"102,383",N


In [37]:
# Resetting index and renaming to create a 'County' column
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.reset_index()
initial_pa_mc_families_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_mc_families_2023.head()


,County,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
1,"Adams County, Pennsylvania",N,N,N,N,N,N,N,N,N,N,N,"110,730",N
2,"Allegheny County, Pennsylvania","229,325",1.0%,0.8%,1.5%,2.8%,5.5%,11.9%,13.7%,23.8%,15.3%,23.6%,"124,087","162,677"
3,"Armstrong County, Pennsylvania",N,N,N,N,N,N,N,N,N,N,N,"87,175",N
4,"Beaver County, Pennsylvania","34,102",1.1%,1.2%,2.7%,3.9%,7.0%,17.9%,13.5%,26.2%,13.0%,13.5%,"102,383",N


In [38]:
# Confirming column data types prior to renaming
print(initial_pa_mc_families_2023.columns)


Index(['County', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='object')


In [39]:
# Converting column data types from integers to strings for renaming
initial_pa_mc_families_2023.columns = initial_pa_mc_families_2023.columns.astype(str)
print(initial_pa_mc_families_2023.columns)


Index(['County', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'],
      dtype='object')


In [40]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_mc_families_2023.replace('N', np.nan, inplace=True)


In [41]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.rename(columns={'2': '# of Married Couple Families', '3': '% Married Couple Families <$10,000', '4': '% Married Couple Families $10,000-$14,999',
                                                                         '5': '% Married Couple Families $15,000-$24,999', '6': '% Married Couple Families $25,000-$34,999', '7': '% Married Couple Families $35,000-$49,999',
                                                                         '8': '% Married Couple Families $50,000-$74,999', '9': '% Married Couple Families $75,000-$99,999',
                                                                         '10': '% Married Couple Families $100,000-$149,999', '11': '% Married Couple Families $150,000-$199,999',
                                                                        '12': '% Married Couple Families $200,000 or More', '13': 'Median Married Couple Families Income ($)',
                                                                       '14': 'Mean Married Couple Families Income ($)'})
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.drop(0)
initial_pa_mc_families_2023 = initial_pa_mc_families_2023.reset_index(drop=True)
initial_pa_mc_families_2023['County'] = initial_pa_mc_families_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_mc_families_2023['County'] = initial_pa_mc_families_2023['County'].apply(lambda x: x.upper())
initial_pa_mc_families_2023.insert(0, 'Year', 2023)
initial_pa_mc_families_2023['% Married Couple Families <$10,000'] = initial_pa_mc_families_2023['% Married Couple Families <$10,000'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $10,000-$14,999'] = initial_pa_mc_families_2023['% Married Couple Families $10,000-$14,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $15,000-$24,999'] = initial_pa_mc_families_2023['% Married Couple Families $15,000-$24,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $25,000-$34,999'] = initial_pa_mc_families_2023['% Married Couple Families $25,000-$34,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $35,000-$49,999'] = initial_pa_mc_families_2023['% Married Couple Families $35,000-$49,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $50,000-$74,999'] = initial_pa_mc_families_2023['% Married Couple Families $50,000-$74,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $75,000-$99,999'] = initial_pa_mc_families_2023['% Married Couple Families $75,000-$99,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $100,000-$149,999'] = initial_pa_mc_families_2023['% Married Couple Families $100,000-$149,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $150,000-$199,999'] = initial_pa_mc_families_2023['% Married Couple Families $150,000-$199,999'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['% Married Couple Families $200,000 or More'] = initial_pa_mc_families_2023['% Married Couple Families $200,000 or More'].str.rstrip('%').astype(float) / 100
initial_pa_mc_families_2023['# of Married Couple Families'] = initial_pa_mc_families_2023['# of Married Couple Families'].str.replace(',', '').astype(float)
initial_pa_mc_families_2023['Median Married Couple Families Income ($)'] = initial_pa_mc_families_2023['Median Married Couple Families Income ($)'].str.replace(',', '').astype(float)
initial_pa_mc_families_2023['Mean Married Couple Families Income ($)'] = initial_pa_mc_families_2023['Mean Married Couple Families Income ($)'].str.replace(',', '').astype(float)
initial_pa_mc_families_2023


,Year,County,# of Married Couple Families,"% Married Couple Families <$10,000","% Married Couple Families $10,000-$14,999","% Married Couple Families $15,000-$24,999","% Married Couple Families $25,000-$34,999","% Married Couple Families $35,000-$49,999","% Married Couple Families $50,000-$74,999","% Married Couple Families $75,000-$99,999","% Married Couple Families $100,000-$149,999","% Married Couple Families $150,000-$199,999","% Married Couple Families $200,000 or More",Median Married Couple Families Income ($),Mean Married Couple Families Income ($)
0,2023,ADAMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110730.0,NaN
1,2023,ALLEGHENY,229325.0,0.010,0.008,0.015,0.028,0.055,0.119,0.137,0.238,0.153,0.236,124087.0,162677.0
2,2023,ARMSTRONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87175.0,NaN
3,2023,BEAVER,34102.0,0.011,0.012,0.027,0.039,0.070,0.179,0.135,0.262,0.130,0.135,102383.0,NaN
4,2023,BERKS,80739.0,0.016,0.009,0.023,0.024,0.085,0.133,0.149,0.260,0.165,0.137,110333.0,132220.0
5,2023,BLAIR,21790.0,0.007,0.019,0.040,0.026,0.104,0.192,0.184,0.252,0.105,0.071,89295.0,NaN
6,2023,BUCKS,145371.0,0.009,0.005,0.021,0.013,0.039,0.106,0.105,0.216,0.202,0.283,144226.0,NaN
7,2023,BUTLER,42556.0,0.007,0.000,0.032,0.025,0.057,0.135,0.116,0.240,0.163,0.225,122564.0,NaN
8,2023,CAMBRIA,25295.0,0.012,0.004,0.029,0.052,0.103,0.180,0.160,0.232,0.113,0.115,91846.0,NaN
9,2023,CARBON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91552.0,NaN


**Income (Nonfamily Households) Data Extraction, Transformation & Loading (2023)**
-
-----------

In [42]:
# Reading in 2023 U.S. Census Bureau income (Nonfamily Households) dataset
initial_pa_nonfamily_2023 = pd.read_excel("Resources/PA_Income_Nonfamily_Households_2023.xlsx")
initial_pa_nonfamily_2023.head(19)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,...,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households,Nonfamily households
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Total,"13,206","241,264","9,155","28,588","51,850","21,132","70,363","29,910","23,057",...,"18,524","117,270","40,354","13,948","335,275","22,776","10,871","31,877","58,669","61,083"
3,"Less than $10,000",6.3%,10.0%,12.3%,6.6%,10.6%,10.8%,5.9%,9.8%,10.2%,...,6.7%,6.9%,8.9%,10.6%,13.6%,11.6%,8.8%,7.3%,9.6%,7.4%
4,"$10,000 to $14,999",4.0%,5.9%,9.9%,7.5%,5.0%,10.8%,4.8%,4.4%,9.1%,...,3.7%,4.4%,4.1%,8.8%,8.9%,10.1%,7.9%,7.5%,7.3%,5.6%
5,"$15,000 to $24,999",11.2%,10.7%,17.4%,11.8%,13.9%,19.6%,9.5%,12.8%,21.3%,...,14.0%,8.8%,9.5%,15.6%,10.5%,17.8%,24.7%,12.7%,13.0%,9.3%
6,"$25,000 to $34,999",15.2%,10.6%,13.3%,10.8%,12.3%,14.8%,8.6%,9.5%,17.1%,...,13.5%,10.2%,7.6%,14.3%,8.1%,13.2%,15.9%,12.9%,13.8%,12.9%
7,"$35,000 to $49,999",13.5%,14.7%,17.4%,18.3%,14.3%,16.0%,12.4%,16.9%,13.8%,...,19.7%,13.0%,18.6%,14.7%,12.1%,11.9%,13.6%,16.2%,16.5%,16.4%
8,"$50,000 to $74,999",25.3%,18.4%,14.7%,23.8%,19.4%,16.6%,21.8%,19.7%,18.8%,...,19.4%,17.0%,20.9%,20.5%,16.9%,14.0%,16.0%,19.3%,17.7%,22.4%
9,"$75,000 to $99,999",12.2%,11.1%,5.4%,10.3%,10.7%,4.5%,11.5%,10.7%,3.9%,...,8.2%,12.1%,11.0%,7.6%,10.0%,10.5%,8.1%,9.3%,9.3%,11.2%


In [43]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Four groupings = Households, Families, Married-couple families & non family households
# % of groups above that fall into the income categories: Less Than $10,000, $10,000-$14,999, $15,000-$24,999, $25,000-$34,999,
# $35,000-$49,999, $50,000-$74,999, $75,000-$99,999, $100,000-$149,999, $150,000-$199,999, $200,000 or More
# Total count for each grouping
# Median income for each grouping
# Mean income for each grouping
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.iloc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], :]
initial_pa_nonfamily_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
2,Total,"13,206","241,264","9,155","28,588","51,850","21,132","70,363","29,910","23,057",...,"18,524","117,270","40,354","13,948","335,275","22,776","10,871","31,877","58,669","61,083"
3,"Less than $10,000",6.3%,10.0%,12.3%,6.6%,10.6%,10.8%,5.9%,9.8%,10.2%,...,6.7%,6.9%,8.9%,10.6%,13.6%,11.6%,8.8%,7.3%,9.6%,7.4%
4,"$10,000 to $14,999",4.0%,5.9%,9.9%,7.5%,5.0%,10.8%,4.8%,4.4%,9.1%,...,3.7%,4.4%,4.1%,8.8%,8.9%,10.1%,7.9%,7.5%,7.3%,5.6%
5,"$15,000 to $24,999",11.2%,10.7%,17.4%,11.8%,13.9%,19.6%,9.5%,12.8%,21.3%,...,14.0%,8.8%,9.5%,15.6%,10.5%,17.8%,24.7%,12.7%,13.0%,9.3%
6,"$25,000 to $34,999",15.2%,10.6%,13.3%,10.8%,12.3%,14.8%,8.6%,9.5%,17.1%,...,13.5%,10.2%,7.6%,14.3%,8.1%,13.2%,15.9%,12.9%,13.8%,12.9%
7,"$35,000 to $49,999",13.5%,14.7%,17.4%,18.3%,14.3%,16.0%,12.4%,16.9%,13.8%,...,19.7%,13.0%,18.6%,14.7%,12.1%,11.9%,13.6%,16.2%,16.5%,16.4%
8,"$50,000 to $74,999",25.3%,18.4%,14.7%,23.8%,19.4%,16.6%,21.8%,19.7%,18.8%,...,19.4%,17.0%,20.9%,20.5%,16.9%,14.0%,16.0%,19.3%,17.7%,22.4%
9,"$75,000 to $99,999",12.2%,11.1%,5.4%,10.3%,10.7%,4.5%,11.5%,10.7%,3.9%,...,8.2%,12.1%,11.0%,7.6%,10.0%,10.5%,8.1%,9.3%,9.3%,11.2%
10,"$100,000 to $149,999",9.9%,10.6%,7.2%,8.3%,8.9%,4.7%,14.6%,9.2%,5.2%,...,10.4%,15.4%,12.7%,6.0%,10.4%,9.0%,3.5%,8.9%,8.8%,9.8%
11,"$150,000 to $199,999",1.9%,4.0%,1.4%,1.6%,2.6%,1.5%,5.1%,4.3%,0.3%,...,2.7%,5.5%,3.6%,2.0%,4.9%,1.5%,0.8%,3.0%,2.2%,2.9%


In [44]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.T
initial_pa_nonfamily_2023.head()


,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
"Adams County, Pennsylvania","13,206",6.3%,4.0%,11.2%,15.2%,13.5%,25.3%,12.2%,9.9%,1.9%,0.4%,"49,817","55,623"
"Allegheny County, Pennsylvania","241,264",10.0%,5.9%,10.7%,10.6%,14.7%,18.4%,11.1%,10.6%,4.0%,3.8%,"47,284","65,779"
"Armstrong County, Pennsylvania","9,155",12.3%,9.9%,17.4%,13.3%,17.4%,14.7%,5.4%,7.2%,1.4%,1.0%,"31,901","46,758"
"Beaver County, Pennsylvania","28,588",6.6%,7.5%,11.8%,10.8%,18.3%,23.8%,10.3%,8.3%,1.6%,0.9%,"43,863","53,163"


In [45]:
# Resetting index and renaming to create a 'County' column
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.reset_index()
initial_pa_nonfamily_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_nonfamily_2023.head()


,County,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Unnamed: 0,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
1,"Adams County, Pennsylvania","13,206",6.3%,4.0%,11.2%,15.2%,13.5%,25.3%,12.2%,9.9%,1.9%,0.4%,"49,817","55,623"
2,"Allegheny County, Pennsylvania","241,264",10.0%,5.9%,10.7%,10.6%,14.7%,18.4%,11.1%,10.6%,4.0%,3.8%,"47,284","65,779"
3,"Armstrong County, Pennsylvania","9,155",12.3%,9.9%,17.4%,13.3%,17.4%,14.7%,5.4%,7.2%,1.4%,1.0%,"31,901","46,758"
4,"Beaver County, Pennsylvania","28,588",6.6%,7.5%,11.8%,10.8%,18.3%,23.8%,10.3%,8.3%,1.6%,0.9%,"43,863","53,163"


In [46]:
# Confirming column data types prior to renaming
print(initial_pa_nonfamily_2023.columns)


Index(['County', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='object')


In [47]:
# Converting column data types from integers to strings for renaming
initial_pa_nonfamily_2023.columns = initial_pa_nonfamily_2023.columns.astype(str)
print(initial_pa_nonfamily_2023.columns)


Index(['County', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'],
      dtype='object')


In [48]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_nonfamily_2023.replace('N', np.nan, inplace=True)


In [49]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.rename(columns={'2': '# of Nonfamily Households', '3': '% Nonfamily Households <$10,000', '4': '% Nonfamily Households $10,000-$14,999',
                                                                         '5': '% Nonfamily Households $15,000-$24,999', '6': '% Nonfamily Households $25,000-$34,999', '7': '% Nonfamily Households $35,000-$49,999',
                                                                         '8': '% Nonfamily Households $50,000-$74,999', '9': '% Nonfamily Households $75,000-$99,999',
                                                                         '10': '% Nonfamily Households $100,000-$149,999', '11': '% Nonfamily Households $150,000-$199,999',
                                                                        '12': '% Nonfamily Households $200,000 or More', '13': 'Median Nonfamily Households Income ($)',
                                                                       '14': 'Mean Nonfamily Households Income ($)'})
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.drop(0)
initial_pa_nonfamily_2023 = initial_pa_nonfamily_2023.reset_index(drop=True)
initial_pa_nonfamily_2023['County'] = initial_pa_nonfamily_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_nonfamily_2023['County'] = initial_pa_nonfamily_2023['County'].apply(lambda x: x.upper())
initial_pa_nonfamily_2023.insert(0, 'Year', 2023)
initial_pa_nonfamily_2023['% Nonfamily Households <$10,000'] = initial_pa_nonfamily_2023['% Nonfamily Households <$10,000'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $10,000-$14,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $10,000-$14,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $15,000-$24,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $15,000-$24,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $25,000-$34,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $25,000-$34,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $35,000-$49,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $35,000-$49,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $50,000-$74,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $50,000-$74,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $75,000-$99,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $75,000-$99,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $100,000-$149,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $100,000-$149,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $150,000-$199,999'] = initial_pa_nonfamily_2023['% Nonfamily Households $150,000-$199,999'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['% Nonfamily Households $200,000 or More'] = initial_pa_nonfamily_2023['% Nonfamily Households $200,000 or More'].str.rstrip('%').astype(float) / 100
initial_pa_nonfamily_2023['# of Nonfamily Households'] = initial_pa_nonfamily_2023['# of Nonfamily Households'].str.replace(',', '').astype(float)
initial_pa_nonfamily_2023['Median Nonfamily Households Income ($)'] = initial_pa_nonfamily_2023['Median Nonfamily Households Income ($)'].str.replace(',', '').astype(float)
initial_pa_nonfamily_2023['Mean Nonfamily Households Income ($)'] = initial_pa_nonfamily_2023['Mean Nonfamily Households Income ($)'].str.replace(',', '').astype(float)
initial_pa_nonfamily_2023


,Year,County,# of Nonfamily Households,"% Nonfamily Households <$10,000","% Nonfamily Households $10,000-$14,999","% Nonfamily Households $15,000-$24,999","% Nonfamily Households $25,000-$34,999","% Nonfamily Households $35,000-$49,999","% Nonfamily Households $50,000-$74,999","% Nonfamily Households $75,000-$99,999","% Nonfamily Households $100,000-$149,999","% Nonfamily Households $150,000-$199,999","% Nonfamily Households $200,000 or More",Median Nonfamily Households Income ($),Mean Nonfamily Households Income ($)
0,2023,ADAMS,13206.0,0.063,0.040,0.112,0.152,0.135,0.253,0.122,0.099,0.019,0.004,49817.0,55623.0
1,2023,ALLEGHENY,241264.0,0.100,0.059,0.107,0.106,0.147,0.184,0.111,0.106,0.040,0.038,47284.0,65779.0
2,2023,ARMSTRONG,9155.0,0.123,0.099,0.174,0.133,0.174,0.147,0.054,0.072,0.014,0.010,31901.0,46758.0
3,2023,BEAVER,28588.0,0.066,0.075,0.118,0.108,0.183,0.238,0.103,0.083,0.016,0.009,43863.0,53163.0
4,2023,BERKS,51850.0,0.106,0.050,0.139,0.123,0.143,0.194,0.107,0.089,0.026,0.023,42344.0,55356.0
5,2023,BLAIR,21132.0,0.108,0.108,0.196,0.148,0.160,0.166,0.045,0.047,0.015,0.008,30525.0,42877.0
6,2023,BUCKS,70363.0,0.059,0.048,0.095,0.086,0.124,0.218,0.115,0.146,0.051,0.057,60183.0,76698.0
7,2023,BUTLER,29910.0,0.098,0.044,0.128,0.095,0.169,0.197,0.107,0.092,0.043,0.027,46163.0,59013.0
8,2023,CAMBRIA,23057.0,0.102,0.091,0.213,0.171,0.138,0.188,0.039,0.052,0.003,0.003,28897.0,39085.0
9,2023,CARBON,10679.0,0.112,0.047,0.173,0.158,0.134,0.174,0.086,0.087,0.026,0.003,35498.0,49776.0


**Auxillary (Total)  Data Extraction, Transformation & Loading (2023)**
-
-----------

In [50]:
# Reading in 2023 U.S. Census Bureau auxillary (Total) dataset
initial_pa_total_aux_2023 = pd.read_excel("Resources/PA_Total_Auxillary_2023.xlsx")
initial_pa_total_aux_2023.head(60)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,...,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent,Percent
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Civilian population 18 years and over,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,PERIOD OF SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gulf War (9/2001 or later) veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
5,Gulf War (8/1990 to 8/2001) veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
6,Vietnam era veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
7,Korean War veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
8,World War II veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
9,SEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Three groupings = Total Population, Veteran Population, Nonveteran Population
# % of population groups above that fall into these age categories: 18 to 34, 35 to 54, 55 to 64, 65 to 74 and 75 & Over
# Labor force participation rate
# Unemployment rate
# % of Population group with income below poverty level in the past 12 months
# % of Population group with any disability
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.iloc[[13, 14, 15, 16, 17, 40, 42, 45, 49], :]
initial_pa_total_aux_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
13,18 to 34 years,25.6%,28.4%,21.5%,23.2%,28.1%,25.3%,22.7%,24.4%,23.7%,...,24.6%,25.4%,27.2%,22.0%,35.4%,24.4%,22.2%,23.5%,21.8%,25.7%
14,35 to 54 years,27.8%,30.3%,29.2%,29.4%,31.1%,28.8%,31.8%,31.1%,28.6%,...,30.3%,33.0%,30.2%,31.7%,31.1%,31.4%,29.7%,30.4%,29.2%,32.2%
15,55 to 64 years,18.4%,15.8%,18.9%,18.6%,17.3%,18.1%,19.0%,18.2%,17.2%,...,20.1%,17.0%,16.8%,18.2%,14.2%,17.4%,18.3%,18.1%,18.6%,17.6%
16,65 to 74 years,16.9%,15.1%,17.8%,17.0%,13.7%,16.1%,15.3%,15.5%,18.2%,...,15.1%,13.9%,14.6%,16.0%,11.5%,15.6%,17.3%,16.4%,17.5%,14.4%
17,75 years and over,11.2%,10.5%,12.7%,11.8%,9.8%,11.7%,11.3%,10.8%,12.3%,...,9.9%,10.7%,11.2%,12.0%,7.7%,11.2%,12.5%,11.6%,12.9%,10.1%
40,Labor force participation rate,81.8%,81.1%,77.0%,79.7%,78.8%,74.9%,82.2%,78.1%,75.1%,...,76.1%,82.7%,78.1%,71.8%,77.2%,73.6%,74.7%,80.9%,81.0%,83.5%
42,Unemployment rate,3.2%,3.9%,2.7%,2.6%,5.1%,3.5%,3.6%,4.4%,4.0%,...,6.5%,3.4%,4.5%,2.8%,6.3%,5.8%,5.3%,2.9%,3.7%,3.7%
45,Income in the past 12 months below poverty level,7.6%,10.8%,10.9%,9.7%,10.2%,14.2%,6.4%,8.6%,11.6%,...,8.7%,6.5%,7.7%,11.9%,19.1%,12.5%,10.7%,8.1%,10.1%,7.3%
49,With any disability,15.0%,15.3%,20.0%,18.3%,16.1%,20.6%,13.9%,15.3%,22.5%,...,19.6%,12.6%,13.9%,19.2%,21.0%,21.3%,19.5%,17.4%,17.5%,17.6%


In [52]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.T
initial_pa_total_aux_2023.head()


,13,14,15,16,17,40,42,45,49
Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
"Adams County, Pennsylvania",25.6%,27.8%,18.4%,16.9%,11.2%,81.8%,3.2%,7.6%,15.0%
"Allegheny County, Pennsylvania",28.4%,30.3%,15.8%,15.1%,10.5%,81.1%,3.9%,10.8%,15.3%
"Armstrong County, Pennsylvania",21.5%,29.2%,18.9%,17.8%,12.7%,77.0%,2.7%,10.9%,20.0%
"Beaver County, Pennsylvania",23.2%,29.4%,18.6%,17.0%,11.8%,79.7%,2.6%,9.7%,18.3%


In [53]:
# Resetting index and renaming to create a 'County' column
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.reset_index()
initial_pa_total_aux_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_total_aux_2023.head()


,County,13,14,15,16,17,40,42,45,49
0,Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
1,"Adams County, Pennsylvania",25.6%,27.8%,18.4%,16.9%,11.2%,81.8%,3.2%,7.6%,15.0%
2,"Allegheny County, Pennsylvania",28.4%,30.3%,15.8%,15.1%,10.5%,81.1%,3.9%,10.8%,15.3%
3,"Armstrong County, Pennsylvania",21.5%,29.2%,18.9%,17.8%,12.7%,77.0%,2.7%,10.9%,20.0%
4,"Beaver County, Pennsylvania",23.2%,29.4%,18.6%,17.0%,11.8%,79.7%,2.6%,9.7%,18.3%


In [54]:
# Confirming column data types prior to renaming
print(initial_pa_total_aux_2023.columns)


Index(['County', 13, 14, 15, 16, 17, 40, 42, 45, 49], dtype='object')


In [55]:
# Converting column data types from integers to strings for renaming
initial_pa_total_aux_2023.columns = initial_pa_total_aux_2023.columns.astype(str)
print(initial_pa_total_aux_2023.columns)


Index(['County', '13', '14', '15', '16', '17', '40', '42', '45', '49'], dtype='object')


In [56]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_total_aux_2023.replace('N', np.nan, inplace=True)


In [57]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.rename(columns={'13': '% Total Population 18-34 Years Old', '14': '% Total Population 35-54 Years Old', '15': '% Total Population 55-64 Years Old',
                                                                         '16': '% Total Population 65-74 Years Old', '17': '% Total Population 75 Years Old & Over', '40': 'Total Labor Force Participation Rate (%)',
                                                                         '42': 'Total Unemployment Rate (%)', '45': '% Total Population With Income Below Poverty Level (Past 12 Months)',
                                                                         '49': '% Total Population With Any Disability'})
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.drop(0)
initial_pa_total_aux_2023 = initial_pa_total_aux_2023.reset_index(drop=True)
initial_pa_total_aux_2023['County'] = initial_pa_total_aux_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_total_aux_2023['County'] = initial_pa_total_aux_2023['County'].apply(lambda x: x.upper())
initial_pa_total_aux_2023.insert(0, 'Year', 2023)
initial_pa_total_aux_2023['% Total Population 18-34 Years Old'] = initial_pa_total_aux_2023['% Total Population 18-34 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population 35-54 Years Old'] = initial_pa_total_aux_2023['% Total Population 35-54 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population 55-64 Years Old'] = initial_pa_total_aux_2023['% Total Population 55-64 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population 65-74 Years Old'] = initial_pa_total_aux_2023['% Total Population 65-74 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population 75 Years Old & Over'] = initial_pa_total_aux_2023['% Total Population 75 Years Old & Over'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['Total Labor Force Participation Rate (%)'] = initial_pa_total_aux_2023['Total Labor Force Participation Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['Total Unemployment Rate (%)'] = initial_pa_total_aux_2023['Total Unemployment Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population With Income Below Poverty Level (Past 12 Months)'] = initial_pa_total_aux_2023['% Total Population With Income Below Poverty Level (Past 12 Months)'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023['% Total Population With Any Disability'] = initial_pa_total_aux_2023['% Total Population With Any Disability'].str.rstrip('%').astype(float) / 100
initial_pa_total_aux_2023


,Year,County,% Total Population 18-34 Years Old,% Total Population 35-54 Years Old,% Total Population 55-64 Years Old,% Total Population 65-74 Years Old,% Total Population 75 Years Old & Over,Total Labor Force Participation Rate (%),Total Unemployment Rate (%),% Total Population With Income Below Poverty Level (Past 12 Months),% Total Population With Any Disability
0,2023,ADAMS,0.256,0.278,0.184,0.169,0.112,0.818,0.032,0.076,0.150
1,2023,ALLEGHENY,0.284,0.303,0.158,0.151,0.105,0.811,0.039,0.108,0.153
2,2023,ARMSTRONG,0.215,0.292,0.189,0.178,0.127,0.770,0.027,0.109,0.200
3,2023,BEAVER,0.232,0.294,0.186,0.170,0.118,0.797,0.026,0.097,0.183
4,2023,BERKS,0.281,0.311,0.173,0.137,0.098,0.788,0.051,0.102,0.161
5,2023,BLAIR,0.253,0.288,0.181,0.161,0.117,0.749,0.035,0.142,0.206
6,2023,BUCKS,0.227,0.318,0.190,0.153,0.113,0.822,0.036,0.064,0.139
7,2023,BUTLER,0.244,0.311,0.182,0.155,0.108,0.781,0.044,0.086,0.153
8,2023,CAMBRIA,0.237,0.286,0.172,0.182,0.123,0.751,0.040,0.116,0.225
9,2023,CARBON,0.221,0.303,0.185,0.179,0.112,0.781,0.063,0.111,0.204


**Auxillary (Veterans)  Data Extraction, Transformation & Loading (2023)**
-
-----------

In [58]:
# Reading in 2023 U.S. Census Bureau auxillary (Veterans) dataset
initial_pa_veterans_aux_2023 = pd.read_excel("Resources/PA_Veterans_Auxillary_2023.xlsx")
initial_pa_veterans_aux_2023.head(60)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,...,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans,Percent Veterans
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Civilian population 18 years and over,9.4%,5.6%,8.9%,7.7%,5.5%,9.0%,5.5%,6.5%,8.1%,...,5.9%,4.6%,6.3%,9.0%,3.5%,6.7%,6.8%,6.3%,7.5%,6.9%
3,PERIOD OF SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gulf War (9/2001 or later) veterans,N,19.1%,N,N,23.0%,N,16.4%,15.8%,N,...,N,17.7%,12.5%,N,22.9%,N,N,N,18.9%,28.2%
5,Gulf War (8/1990 to 8/2001) veterans,N,16.3%,N,N,13.8%,N,17.7%,22.3%,N,...,N,16.8%,13.7%,N,15.4%,N,N,N,17.3%,20.9%
6,Vietnam era veterans,N,37.5%,N,N,33.4%,N,35.6%,39.0%,N,...,N,37.4%,45.1%,N,29.0%,N,N,N,40.7%,30.7%
7,Korean War veterans,N,6.2%,N,N,7.0%,N,3.8%,3.8%,N,...,N,6.8%,3.2%,N,3.8%,N,N,N,6.7%,3.3%
8,World War II veterans,N,1.3%,N,N,0.5%,N,0.7%,1.8%,N,...,N,1.8%,0.7%,N,0.5%,N,N,N,1.4%,1.8%
9,SEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Three groupings = Total Population, Veteran Population, Nonveteran Population
# % of population groups above that fall into these age categories: 18 to 34, 35 to 54, 55 to 64, 65 to 74 and 75 & Over
# Labor force participation rate
# Unemployment rate
# % of Population group with income below poverty level in the past 12 months
# % of Population group with any disability
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.iloc[[13, 14, 15, 16, 17, 40, 42, 45, 49], :]
initial_pa_veterans_aux_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
13,18 to 34 years,10.0%,5.9%,5.8%,6.3%,6.4%,7.0%,6.7%,3.3%,4.1%,...,3.8%,4.2%,3.6%,8.3%,5.6%,5.4%,5.0%,5.9%,3.2%,7.6%
14,35 to 54 years,18.2%,18.6%,13.4%,19.8%,19.5%,21.9%,14.3%,14.9%,25.9%,...,17.5%,18.3%,11.5%,19.1%,21.2%,19.4%,19.9%,24.1%,18.8%,22.3%
15,55 to 64 years,20.8%,17.4%,16.7%,22.2%,17.4%,28.7%,16.5%,23.0%,19.0%,...,24.7%,13.8%,19.7%,27.9%,22.0%,14.8%,24.4%,17.5%,17.3%,21.1%
16,65 to 74 years,24.3%,22.3%,26.8%,20.9%,21.6%,15.5%,21.5%,25.2%,18.5%,...,21.5%,23.1%,24.2%,15.5%,26.5%,22.5%,17.5%,13.9%,24.7%,20.8%
17,75 years and over,26.8%,35.9%,37.3%,30.8%,35.1%,26.9%,41.0%,33.6%,32.5%,...,32.5%,40.5%,40.9%,29.3%,24.7%,37.8%,33.2%,38.6%,36.0%,28.2%
40,Labor force participation rate,77.0%,81.6%,78.7%,72.8%,87.7%,64.8%,78.4%,77.6%,78.7%,...,54.1%,79.4%,67.3%,73.2%,67.7%,74.1%,81.6%,89.3%,83.9%,83.1%
42,Unemployment rate,0.0%,3.1%,2.5%,0.0%,2.4%,8.7%,1.4%,7.0%,1.3%,...,0.0%,1.1%,4.1%,2.7%,3.0%,7.2%,2.8%,0.8%,3.3%,5.5%
45,Income in the past 12 months below poverty level,4.9%,7.5%,13.9%,6.4%,7.1%,9.7%,3.6%,7.2%,7.6%,...,5.3%,7.8%,5.5%,9.1%,17.8%,10.3%,8.1%,7.8%,9.3%,4.1%
49,With any disability,23.3%,29.4%,34.4%,29.4%,25.3%,38.7%,30.4%,27.5%,31.6%,...,34.9%,25.4%,27.0%,41.3%,36.1%,33.8%,24.3%,30.8%,28.0%,31.4%


In [60]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.T
initial_pa_veterans_aux_2023.head()


,13,14,15,16,17,40,42,45,49
Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
"Adams County, Pennsylvania",10.0%,18.2%,20.8%,24.3%,26.8%,77.0%,0.0%,4.9%,23.3%
"Allegheny County, Pennsylvania",5.9%,18.6%,17.4%,22.3%,35.9%,81.6%,3.1%,7.5%,29.4%
"Armstrong County, Pennsylvania",5.8%,13.4%,16.7%,26.8%,37.3%,78.7%,2.5%,13.9%,34.4%
"Beaver County, Pennsylvania",6.3%,19.8%,22.2%,20.9%,30.8%,72.8%,0.0%,6.4%,29.4%


In [61]:
# Resetting index and renaming to create a 'County' column
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.reset_index()
initial_pa_veterans_aux_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_veterans_aux_2023.head()


,County,13,14,15,16,17,40,42,45,49
0,Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
1,"Adams County, Pennsylvania",10.0%,18.2%,20.8%,24.3%,26.8%,77.0%,0.0%,4.9%,23.3%
2,"Allegheny County, Pennsylvania",5.9%,18.6%,17.4%,22.3%,35.9%,81.6%,3.1%,7.5%,29.4%
3,"Armstrong County, Pennsylvania",5.8%,13.4%,16.7%,26.8%,37.3%,78.7%,2.5%,13.9%,34.4%
4,"Beaver County, Pennsylvania",6.3%,19.8%,22.2%,20.9%,30.8%,72.8%,0.0%,6.4%,29.4%


In [62]:
# Confirming column data types prior to renaming
print(initial_pa_veterans_aux_2023.columns)


Index(['County', 13, 14, 15, 16, 17, 40, 42, 45, 49], dtype='object')


In [63]:
# Converting column data types from integers to strings for renaming
initial_pa_veterans_aux_2023.columns = initial_pa_veterans_aux_2023.columns.astype(str)
print(initial_pa_veterans_aux_2023.columns)


Index(['County', '13', '14', '15', '16', '17', '40', '42', '45', '49'], dtype='object')


In [64]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_veterans_aux_2023.replace('N', np.nan, inplace=True)


In [65]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.rename(columns={'13': '% Veteran Population 18-34 Years Old', '14': '% Veteran Population 35-54 Years Old', '15': '% Veteran Population 55-64 Years Old',
                                                                         '16': '% Veteran Population 65-74 Years Old', '17': '% Veteran Population 75 Years Old & Over', '40': 'Veteran Labor Force Participation Rate (%)',
                                                                         '42': 'Veteran Unemployment Rate (%)', '45': '% Veteran Population With Income Below Poverty Level (Past 12 Months)',
                                                                         '49': '% Veteran Population With Any Disability'})
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.drop(0)
initial_pa_veterans_aux_2023 = initial_pa_veterans_aux_2023.reset_index(drop=True)
initial_pa_veterans_aux_2023['County'] = initial_pa_veterans_aux_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_veterans_aux_2023['County'] = initial_pa_veterans_aux_2023['County'].apply(lambda x: x.upper())
initial_pa_veterans_aux_2023.insert(0, 'Year', 2023)
initial_pa_veterans_aux_2023['% Veteran Population 18-34 Years Old'] = initial_pa_veterans_aux_2023['% Veteran Population 18-34 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population 35-54 Years Old'] = initial_pa_veterans_aux_2023['% Veteran Population 35-54 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population 55-64 Years Old'] = initial_pa_veterans_aux_2023['% Veteran Population 55-64 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population 65-74 Years Old'] = initial_pa_veterans_aux_2023['% Veteran Population 65-74 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population 75 Years Old & Over'] = initial_pa_veterans_aux_2023['% Veteran Population 75 Years Old & Over'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['Veteran Labor Force Participation Rate (%)'] = initial_pa_veterans_aux_2023['Veteran Labor Force Participation Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['Veteran Unemployment Rate (%)'] = initial_pa_veterans_aux_2023['Veteran Unemployment Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population With Income Below Poverty Level (Past 12 Months)'] = initial_pa_veterans_aux_2023['% Veteran Population With Income Below Poverty Level (Past 12 Months)'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023['% Veteran Population With Any Disability'] = initial_pa_veterans_aux_2023['% Veteran Population With Any Disability'].str.rstrip('%').astype(float) / 100
initial_pa_veterans_aux_2023


,Year,County,% Veteran Population 18-34 Years Old,% Veteran Population 35-54 Years Old,% Veteran Population 55-64 Years Old,% Veteran Population 65-74 Years Old,% Veteran Population 75 Years Old & Over,Veteran Labor Force Participation Rate (%),Veteran Unemployment Rate (%),% Veteran Population With Income Below Poverty Level (Past 12 Months),% Veteran Population With Any Disability
0,2023,ADAMS,0.100,0.182,0.208,0.243,0.268,0.770,0.000,0.049,0.233
1,2023,ALLEGHENY,0.059,0.186,0.174,0.223,0.359,0.816,0.031,0.075,0.294
2,2023,ARMSTRONG,0.058,0.134,0.167,0.268,0.373,0.787,0.025,0.139,0.344
3,2023,BEAVER,0.063,0.198,0.222,0.209,0.308,0.728,0.000,0.064,0.294
4,2023,BERKS,0.064,0.195,0.174,0.216,0.351,0.877,0.024,0.071,0.253
5,2023,BLAIR,0.070,0.219,0.287,0.155,0.269,0.648,0.087,0.097,0.387
6,2023,BUCKS,0.067,0.143,0.165,0.215,0.410,0.784,0.014,0.036,0.304
7,2023,BUTLER,0.033,0.149,0.230,0.252,0.336,0.776,0.070,0.072,0.275
8,2023,CAMBRIA,0.041,0.259,0.190,0.185,0.325,0.787,0.013,0.076,0.316
9,2023,CARBON,0.169,0.231,0.138,0.174,0.288,0.894,0.033,0.034,0.250


**Auxillary (Nonveterans)  Data Extraction, Transformation & Loading (2023)**
-
-----------

In [66]:
# Reading in 2023 U.S. Census Bureau auxillary (Nonveterans) dataset
initial_pa_nonveterans_aux_2023 = pd.read_excel("Resources/PA_Nonveterans_Auxillary_2023.xlsx")
initial_pa_nonveterans_aux_2023.head(60)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,...,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans,Percent Nonveterans
1,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
2,Civilian population 18 years and over,90.6%,94.4%,91.1%,92.3%,94.5%,91.0%,94.5%,93.5%,91.9%,...,94.1%,95.4%,93.7%,91.0%,96.5%,93.3%,93.2%,93.7%,92.5%,93.1%
3,PERIOD OF SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gulf War (9/2001 or later) veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
5,Gulf War (8/1990 to 8/2001) veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
6,Vietnam era veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
7,Korean War veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
8,World War II veterans,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
9,SEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Three groupings = Total Population, Veteran Population, Nonveteran Population
# % of population groups above that fall into these age categories: 18 to 34, 35 to 54, 55 to 64, 65 to 74 and 75 & Over
# Labor force participation rate
# Unemployment rate
# % of Population group with income below poverty level in the past 12 months
# % of Population group with any disability
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.iloc[[13, 14, 15, 16, 17, 40, 42, 45, 49], :]
initial_pa_nonveterans_aux_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
13,18 to 34 years,27.3%,29.7%,23.0%,24.6%,29.3%,27.2%,23.6%,25.9%,25.4%,...,25.9%,26.4%,28.8%,23.4%,36.5%,25.7%,23.5%,24.7%,23.3%,27.0%
14,35 to 54 years,28.8%,31.0%,30.7%,30.2%,31.8%,29.5%,32.8%,32.2%,28.8%,...,31.1%,33.7%,31.4%,33.0%,31.5%,32.3%,30.4%,30.8%,30.1%,33.0%
15,55 to 64 years,18.2%,15.7%,19.1%,18.3%,17.3%,17.0%,19.1%,17.9%,17.1%,...,19.8%,17.2%,16.6%,17.3%,13.9%,17.6%,17.9%,18.1%,18.7%,17.3%
16,65 to 74 years,16.2%,14.6%,16.9%,16.7%,13.2%,16.1%,14.9%,14.8%,18.2%,...,14.7%,13.4%,13.9%,16.1%,11.0%,15.1%,17.3%,16.5%,17.0%,14.0%
17,75 years and over,9.6%,9.0%,10.3%,10.2%,8.4%,10.2%,9.6%,9.2%,10.5%,...,8.5%,9.2%,9.2%,10.3%,7.1%,9.3%,11.0%,9.7%,11.0%,8.7%
40,Labor force participation rate,82.1%,81.1%,76.9%,80.1%,78.6%,75.7%,82.3%,78.1%,74.8%,...,77.0%,82.8%,78.4%,71.6%,77.4%,73.6%,74.3%,80.5%,80.9%,83.5%
42,Unemployment rate,3.5%,3.9%,2.7%,2.7%,5.2%,3.2%,3.6%,4.3%,4.2%,...,6.7%,3.4%,4.5%,2.8%,6.4%,5.7%,5.4%,3.0%,3.7%,3.6%
45,Income in the past 12 months below poverty level,7.9%,11.0%,10.6%,10.0%,10.4%,14.7%,6.6%,8.7%,12.0%,...,8.9%,6.5%,7.8%,12.2%,19.1%,12.6%,10.8%,8.1%,10.2%,7.5%
49,With any disability,14.1%,14.4%,18.6%,17.4%,15.5%,18.9%,12.9%,14.5%,21.6%,...,18.6%,12.0%,13.0%,16.9%,20.5%,20.4%,19.2%,16.5%,16.6%,16.6%


In [68]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.T
initial_pa_nonveterans_aux_2023.head()


,13,14,15,16,17,40,42,45,49
Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
"Adams County, Pennsylvania",27.3%,28.8%,18.2%,16.2%,9.6%,82.1%,3.5%,7.9%,14.1%
"Allegheny County, Pennsylvania",29.7%,31.0%,15.7%,14.6%,9.0%,81.1%,3.9%,11.0%,14.4%
"Armstrong County, Pennsylvania",23.0%,30.7%,19.1%,16.9%,10.3%,76.9%,2.7%,10.6%,18.6%
"Beaver County, Pennsylvania",24.6%,30.2%,18.3%,16.7%,10.2%,80.1%,2.7%,10.0%,17.4%


In [69]:
# Resetting index and renaming to create a 'County' column
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.reset_index()
initial_pa_nonveterans_aux_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_nonveterans_aux_2023.head()


,County,13,14,15,16,17,40,42,45,49
0,Unnamed: 0,18 to 34 years,35 to 54 years,55 to 64 years,65 to 74 years,75 years and over,Labor force participation rate,Unemployment rate,Income in the past 12 months below poverty level,With any disability
1,"Adams County, Pennsylvania",27.3%,28.8%,18.2%,16.2%,9.6%,82.1%,3.5%,7.9%,14.1%
2,"Allegheny County, Pennsylvania",29.7%,31.0%,15.7%,14.6%,9.0%,81.1%,3.9%,11.0%,14.4%
3,"Armstrong County, Pennsylvania",23.0%,30.7%,19.1%,16.9%,10.3%,76.9%,2.7%,10.6%,18.6%
4,"Beaver County, Pennsylvania",24.6%,30.2%,18.3%,16.7%,10.2%,80.1%,2.7%,10.0%,17.4%


In [70]:
# Confirming column data types prior to renaming
print(initial_pa_nonveterans_aux_2023.columns)


Index(['County', 13, 14, 15, 16, 17, 40, 42, 45, 49], dtype='object')


In [71]:
# Converting column data types from integers to strings for renaming
initial_pa_nonveterans_aux_2023.columns = initial_pa_nonveterans_aux_2023.columns.astype(str)
print(initial_pa_nonveterans_aux_2023.columns)


Index(['County', '13', '14', '15', '16', '17', '40', '42', '45', '49'], dtype='object')


In [72]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_nonveterans_aux_2023.replace('N', np.nan, inplace=True)


In [73]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges and all % data types are coverted to floats
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.rename(columns={'13': '% Nonveteran Population 18-34 Years Old', '14': '% Nonveteran Population 35-54 Years Old', '15': '% Nonveteran Population 55-64 Years Old',
                                                                         '16': '% Nonveteran Population 65-74 Years Old', '17': '% Nonveteran Population 75 Years Old & Over', '40': 'Nonveteran Labor Force Participation Rate (%)',
                                                                         '42': 'Nonveteran Unemployment Rate (%)', '45': '% Nonveteran Population With Income Below Poverty Level (Past 12 Months)',
                                                                         '49': '% Nonveteran Population With Any Disability'})
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.drop(0)
initial_pa_nonveterans_aux_2023 = initial_pa_nonveterans_aux_2023.reset_index(drop=True)
initial_pa_nonveterans_aux_2023['County'] = initial_pa_nonveterans_aux_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_nonveterans_aux_2023['County'] = initial_pa_nonveterans_aux_2023['County'].apply(lambda x: x.upper())
initial_pa_nonveterans_aux_2023.insert(0, 'Year', 2023)
initial_pa_nonveterans_aux_2023['% Nonveteran Population 18-34 Years Old'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population 18-34 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population 35-54 Years Old'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population 35-54 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population 55-64 Years Old'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population 55-64 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population 65-74 Years Old'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population 65-74 Years Old'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population 75 Years Old & Over'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population 75 Years Old & Over'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['Nonveteran Labor Force Participation Rate (%)'] = initial_pa_nonveterans_aux_2023['Nonveteran Labor Force Participation Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['Nonveteran Unemployment Rate (%)'] = initial_pa_nonveterans_aux_2023['Nonveteran Unemployment Rate (%)'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population With Income Below Poverty Level (Past 12 Months)'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population With Income Below Poverty Level (Past 12 Months)'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023['% Nonveteran Population With Any Disability'] = initial_pa_nonveterans_aux_2023['% Nonveteran Population With Any Disability'].str.rstrip('%').astype(float) / 100
initial_pa_nonveterans_aux_2023


,Year,County,% Nonveteran Population 18-34 Years Old,% Nonveteran Population 35-54 Years Old,% Nonveteran Population 55-64 Years Old,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability
0,2023,ADAMS,0.273,0.288,0.182,0.162,0.096,0.821,0.035,0.079,0.141
1,2023,ALLEGHENY,0.297,0.310,0.157,0.146,0.090,0.811,0.039,0.110,0.144
2,2023,ARMSTRONG,0.230,0.307,0.191,0.169,0.103,0.769,0.027,0.106,0.186
3,2023,BEAVER,0.246,0.302,0.183,0.167,0.102,0.801,0.027,0.100,0.174
4,2023,BERKS,0.293,0.318,0.173,0.132,0.084,0.786,0.052,0.104,0.155
5,2023,BLAIR,0.272,0.295,0.170,0.161,0.102,0.757,0.032,0.147,0.189
6,2023,BUCKS,0.236,0.328,0.191,0.149,0.096,0.823,0.036,0.066,0.129
7,2023,BUTLER,0.259,0.322,0.179,0.148,0.092,0.781,0.043,0.087,0.145
8,2023,CAMBRIA,0.254,0.288,0.171,0.182,0.105,0.748,0.042,0.120,0.216
9,2023,CARBON,0.226,0.310,0.189,0.180,0.096,0.773,0.065,0.118,0.200


**Fertility & Income Assistance Data Extraction, Transformation & Loading (2023)**
-
-----------

In [74]:
# Reading in 2023 U.S. Census Bureau Fertility & Income Assistance dataset
initial_pa_fertility_2023 = pd.read_excel("Resources/PA_Fertility_2023.xlsx")
initial_pa_fertility_2023.head(60)


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
0,NaN,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,...,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months,Women with births in the past 12 months
1,NaN,"Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women",...,"Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women","Rate per 1,000 women"
2,Label,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,...,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate,Estimate
3,Women 15 to 50 years,60,51,57,40,62,48,46,47,60,...,55,58,41,69,49,57,59,58,62,45
4,15 to 19 years,0,9,30,0,6,0,0,0,7,...,0,2,0,0,14,0,61,0,19,0
5,20 to 34 years,94,79,123,67,102,64,84,82,107,...,92,82,63,136,69,110,106,74,108,91
6,35 to 50 years,49,35,9,26,45,49,30,34,39,...,40,54,34,32,35,33,18,61,35,22
7,RACE AND HISPANIC OR LATINO ORIGIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,One race,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
9,White,54,44,59,46,57,51,46,47,69,...,56,59,34,78,44,45,62,60,65,47


In [75]:
# Retrieving only rows that contain the desired county summary demographic statistics
# Primary grouping = Population of women with births in the past 12 months:
# Rate per 1,000 Women (within above group) in the following age categories: 15-19 Years Old, 20-34 Years Old, 35-50 Years Old 
# Rate per 1,000 Women (within above group) Received Public Assistance Income
initial_pa_fertility_2023 = initial_pa_fertility_2023.iloc[[4, 5, 6, 37], :]
initial_pa_fertility_2023


,Unnamed: 0,"Adams County, Pennsylvania","Allegheny County, Pennsylvania","Armstrong County, Pennsylvania","Beaver County, Pennsylvania","Berks County, Pennsylvania","Blair County, Pennsylvania","Bucks County, Pennsylvania","Butler County, Pennsylvania","Cambria County, Pennsylvania",...,"Monroe County, Pennsylvania","Montgomery County, Pennsylvania","Northampton County, Pennsylvania","Northumberland County, Pennsylvania","Philadelphia County, Pennsylvania","Schuylkill County, Pennsylvania","Somerset County, Pennsylvania","Washington County, Pennsylvania","Westmoreland County, Pennsylvania","York County, Pennsylvania"
4,15 to 19 years,0,9,30,0,6,0,0,0,7,...,0,2,0,0,14,0,61,0,19,0
5,20 to 34 years,94,79,123,67,102,64,84,82,107,...,92,82,63,136,69,110,106,74,108,91
6,35 to 50 years,49,35,9,26,45,49,30,34,39,...,40,54,34,32,35,33,18,61,35,22
37,Received public assistance income,N,189,68,222,57,0,199,0,27,...,211,131,59,499,93,358,N,96,89,0


In [76]:
# Transposing the DataFrame's format for process integrity and future merging simplicity
initial_pa_fertility_2023 = initial_pa_fertility_2023.T
initial_pa_fertility_2023.head()


,4,5,6,37
Unnamed: 0,15 to 19 years,20 to 34 years,35 to 50 years,Received public assistance income
"Adams County, Pennsylvania",0,94,49,N
"Allegheny County, Pennsylvania",9,79,35,189
"Armstrong County, Pennsylvania",30,123,9,68
"Beaver County, Pennsylvania",0,67,26,222


In [77]:
# Resetting index and renaming to create a 'County' column
initial_pa_fertility_2023 = initial_pa_fertility_2023.reset_index()
initial_pa_fertility_2023.rename(columns={'index': 'County'}, inplace=True)
initial_pa_fertility_2023.head()


,County,4,5,6,37
0,Unnamed: 0,15 to 19 years,20 to 34 years,35 to 50 years,Received public assistance income
1,"Adams County, Pennsylvania",0,94,49,N
2,"Allegheny County, Pennsylvania",9,79,35,189
3,"Armstrong County, Pennsylvania",30,123,9,68
4,"Beaver County, Pennsylvania",0,67,26,222


In [78]:
# Confirming column data types prior to renaming
print(initial_pa_fertility_2023.columns)


Index(['County', 4, 5, 6, 37], dtype='object')


In [79]:
# Converting column data types from integers to strings for renaming
initial_pa_fertility_2023.columns = initial_pa_fertility_2023.columns.astype(str)
print(initial_pa_fertility_2023.columns)


Index(['County', '4', '5', '6', '37'], dtype='object')


In [80]:
# Finding and replacing all 'N' values with 'NaN' across the entire DataFrame for future manipulation
initial_pa_fertility_2023.replace('N', np.nan, inplace=True)


In [81]:
# Further DataFrame cleaning including execution of column renaming, dropping obselete (0) index row and subsequent resetting,
# as well as converting 'County' values to uppercase county names only (removing ' County, Pennsylvania')
# A corresponding year column is also included for future merges
# Converting all values to floats as well
initial_pa_fertility_2023 = initial_pa_fertility_2023.rename(columns={'4': 'Birth Rate Per 1000 Women (15-19 Years Old)', '5': 'Birth Rate Per 1000 Women (20-34 Years Old)', '6': 'Birth Rate Per 1000 Women (35-50 Years Old)',
                                                                         '37': 'Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)'})
initial_pa_fertility_2023 = initial_pa_fertility_2023.drop(0)
initial_pa_fertility_2023 = initial_pa_fertility_2023.reset_index(drop=True)
initial_pa_fertility_2023['County'] = initial_pa_fertility_2023['County'].str.replace(' County, Pennsylvania', '', regex=False)
initial_pa_fertility_2023['County'] = initial_pa_fertility_2023['County'].apply(lambda x: x.upper())
initial_pa_fertility_2023.insert(0, 'Year', 2023)
columns_to_floats = ['Birth Rate Per 1000 Women (15-19 Years Old)', 'Birth Rate Per 1000 Women (20-34 Years Old)',
                     'Birth Rate Per 1000 Women (35-50 Years Old)', 'Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)']
initial_pa_fertility_2023[columns_to_floats] = initial_pa_fertility_2023[columns_to_floats].astype(float)
initial_pa_fertility_2023


,Year,County,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.0,94.0,49.0,NaN
1,2023,ALLEGHENY,9.0,79.0,35.0,189.0
2,2023,ARMSTRONG,30.0,123.0,9.0,68.0
3,2023,BEAVER,0.0,67.0,26.0,222.0
4,2023,BERKS,6.0,102.0,45.0,57.0
5,2023,BLAIR,0.0,64.0,49.0,0.0
6,2023,BUCKS,0.0,84.0,30.0,199.0
7,2023,BUTLER,0.0,82.0,34.0,0.0
8,2023,CAMBRIA,7.0,107.0,39.0,27.0
9,2023,CARBON,NaN,NaN,NaN,NaN


**Combining All 2023 Derived Datasets/DataFrames for County Level Attribute Aggregation**
-
-----------

In [82]:
# Due to all County attribute DataFrames having the same index, we can simply outer merge all DataFrames
# from the same year on 'County' and 'Year' sequentially. This will produce a master DataFrame for year 2023
dfs_2023 = [initial_pa_demographic_2023, initial_pa_education_2023, initial_pa_households_2023, initial_pa_families_2023,
            initial_pa_mc_families_2023, initial_pa_nonfamily_2023, initial_pa_total_aux_2023, initial_pa_veterans_aux_2023,
            initial_pa_nonveterans_aux_2023, initial_pa_fertility_2023]


In [83]:
# 2023 DataFrame merging

# Start with the first DataFrame
merged_dfs_2023 = dfs_2023[0]

# Iteratively merge the remaining DataFrames
for df in dfs_2023[1:]:
    merged_dfs_2023 = pd.merge(merged_dfs_2023, df, on=['County', 'Year'], how='outer')

# Display the final resulting 2023 DataFrame
merged_dfs_2023


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.162,0.096,0.821,0.035,0.079,0.141,0.0,94.0,49.0,NaN
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.146,0.090,0.811,0.039,0.110,0.144,9.0,79.0,35.0,189.0
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.169,0.103,0.769,0.027,0.106,0.186,30.0,123.0,9.0,68.0
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.167,0.102,0.801,0.027,0.100,0.174,0.0,67.0,26.0,222.0
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.132,0.084,0.786,0.052,0.104,0.155,6.0,102.0,45.0,57.0
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.161,0.102,0.757,0.032,0.147,0.189,0.0,64.0,49.0,0.0
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.149,0.096,0.823,0.036,0.066,0.129,0.0,84.0,30.0,199.0
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.148,0.092,0.781,0.043,0.087,0.145,0.0,82.0,34.0,0.0
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.182,0.105,0.748,0.042,0.120,0.216,7.0,107.0,39.0,27.0
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.180,0.096,0.773,0.065,0.118,0.200,NaN,NaN,NaN,NaN


In [84]:
# Due to the Pennsylvania Income Nonfamilies dataset possessing (3) more county entries in 2023, we will drop
# the (3) 'unnamed' rows resulting from our outer merge above
# This will ensure our 2023 Data Array has the proper shape for Neural Network model extrapolation
merged_dfs_2023.drop([40, 41, 42], inplace=True)
merged_dfs_2023


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.162,0.096,0.821,0.035,0.079,0.141,0.0,94.0,49.0,NaN
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.146,0.090,0.811,0.039,0.110,0.144,9.0,79.0,35.0,189.0
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.169,0.103,0.769,0.027,0.106,0.186,30.0,123.0,9.0,68.0
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.167,0.102,0.801,0.027,0.100,0.174,0.0,67.0,26.0,222.0
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.132,0.084,0.786,0.052,0.104,0.155,6.0,102.0,45.0,57.0
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.161,0.102,0.757,0.032,0.147,0.189,0.0,64.0,49.0,0.0
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.149,0.096,0.823,0.036,0.066,0.129,0.0,84.0,30.0,199.0
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.148,0.092,0.781,0.043,0.087,0.145,0.0,82.0,34.0,0.0
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.182,0.105,0.748,0.042,0.120,0.216,7.0,107.0,39.0,27.0
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.180,0.096,0.773,0.065,0.118,0.200,NaN,NaN,NaN,NaN


In [85]:
# Confirming Data Types for our 2023 combined county attribute DataFrame
merged_dfs_2023.dtypes


Year                                                                        int64
County                                                                     object
% Male                                                                    float64
% Female                                                                  float64
% Male > 18 Years Old                                                     float64
                                                                           ...   
% Nonveteran Population With Any Disability                               float64
Birth Rate Per 1000 Women (15-19 Years Old)                               float64
Birth Rate Per 1000 Women (20-34 Years Old)                               float64
Birth Rate Per 1000 Women (35-50 Years Old)                               float64
Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)    float64
Length: 103, dtype: object

In [86]:
# Confirming Data Types for our 2023 combined county attribute DataFrame
merged_dfs_2023.dtypes.tolist()


[dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 

**2023 Combined County Attribute Table Preprocessing and Data Imputation**
-
-----------

In [87]:
# First, exploring DataFrame for missing/'NaN'/Null values (full columns contain 40 values)
non_null_counts = merged_dfs_2023.count()
non_null_counts


Year                                                                      40
County                                                                    40
% Male                                                                    40
% Female                                                                  40
% Male > 18 Years Old                                                     40
                                                                          ..
% Nonveteran Population With Any Disability                               40
Birth Rate Per 1000 Women (15-19 Years Old)                               39
Birth Rate Per 1000 Women (20-34 Years Old)                               39
Birth Rate Per 1000 Women (35-50 Years Old)                               39
Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)    36
Length: 103, dtype: int64

In [88]:
# Describing our DataFrame regarding missing values
columns_without_40_values = non_null_counts[non_null_counts != 40]
count_columns_without_40_values = len(columns_without_40_values)


print(f"Columns without 40 non-null values: ({count_columns_without_40_values} Total)")
print("------------------------------------------")
for column, count in columns_without_40_values.items():
    print(f"{column}: {count}")


Columns without 40 non-null values: (16 Total)
------------------------------------------
# of Married Couple Families: 31
% Married Couple Families <$10,000: 31
% Married Couple Families $10,000-$14,999: 31
% Married Couple Families $15,000-$24,999: 31
% Married Couple Families $25,000-$34,999: 31
% Married Couple Families $35,000-$49,999: 31
% Married Couple Families $50,000-$74,999: 31
% Married Couple Families $75,000-$99,999: 31
% Married Couple Families $100,000-$149,999: 31
% Married Couple Families $150,000-$199,999: 31
% Married Couple Families $200,000 or More: 31
Mean Married Couple Families Income ($): 13
Birth Rate Per 1000 Women (15-19 Years Old): 39
Birth Rate Per 1000 Women (20-34 Years Old): 39
Birth Rate Per 1000 Women (35-50 Years Old): 39
Received Public Assistance Income Rate Per 1000 Women (Past 12 Months): 36


The below imputation strategy for 2023 table missing values is mirrored from the overall combined county election data imputation strategy utilized for aggregate model training dataset inclusive of 2012, 2016 & 2019.

------


It is important to note that this calculation indicates across our dataset/DataFrame, we have (16) columns/county level attributes that possess 1 or more 'NaN' or Null values. Futhermore, from the list of columns/attributes above that fall into this category - we are able to determine how many values are missing respectively and from which original datasets the missing information is derived from:

-S1901: 'Married Couple Families' data

While this table within the above mentioned dataset proves to have the most 'NaN'/Null values contained, the majority of columns missing data are only missing (9) county values (across 40) in regards to the % of Married Couple Families falling into the predefined income brackets. As the entire dataset contains referential data from other counties, we will impute the (9) missing values with Sklearn SimpleImputer and a mean strategy for each column. Although our model will already inlcude these metrics for Families, Households and Nonfamily Households, by maintaining/keeping data that represents an entire population segement surveyed by the U.S. Census Bureau (Married Couple Families), we hope to achieve a more representative county sample size. We will also normalize the associated percentage columns.

The most significant number of 'NaN'/Null values of (27) comes from the 'Mean Married Couple Familes Income ($)' column via this same dataset table. As (27) missing values represents missing data from (27) out of (40) counties AND a mean income metric is included from the other three aforementioned surveyed groups, we will remove this column completely from our dataset/Dataframe rather than imputing (27) values.

We will also Impute the (9) missing values for the '# of Married Couple Families' column via Sklearn SimpleImputer with a mean strategy. Associated column normalization is not required here as values aren't represented as percentages.

-S1301: 'Fertility & Income Assistance' data

The above output shows the final four columns contaning 'NaN'/Null values are 'Birth Rate Per 1000 Women (15-19 Years Old)', 'Birth Rate Per 1000 Women (20-34 Years Old)', 'Birth Rate Per 1000 Women (35-50 Years Old)' & 'Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)' from the Fertility and Income Assistance dataset. Each column contains (1), (1), (1) and (4) missing values for counties respectively. As this is the primary attribute data associated with healthcare AND there's a very miniscule number of missing values, we will Impute via Sklearn SimpleImputer with a mean strategy here as well. Associated column normalization is not required here as values aren't represented as percentages.

In [89]:
# S1901 Imputation % columns
economic_columns_to_impute = ['% Married Couple Families <$10,000', '% Married Couple Families $10,000-$14,999',
                             '% Married Couple Families $15,000-$24,999', '% Married Couple Families $25,000-$34,999',
                             '% Married Couple Families $35,000-$49,999', '% Married Couple Families $50,000-$74,999',
                              '% Married Couple Families $75,000-$99,999', '% Married Couple Families $100,000-$149,999',
                              '% Married Couple Families $150,000-$199,999', '% Married Couple Families $200,000 or More'
                             ]

econmic_percentage_imputer = SimpleImputer(strategy='mean')
merged_dfs_2023[economic_columns_to_impute] = econmic_percentage_imputer.fit_transform(merged_dfs_2023[economic_columns_to_impute])

# Subsequent Normalization for all income bracket % associated columns for Married Couple Families
merged_dfs_2023[economic_columns_to_impute] = merged_dfs_2023[economic_columns_to_impute].div(merged_dfs_2023[economic_columns_to_impute].sum(axis=1), axis=0)

# Display
merged_dfs_2023


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.162,0.096,0.821,0.035,0.079,0.141,0.0,94.0,49.0,NaN
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.146,0.090,0.811,0.039,0.110,0.144,9.0,79.0,35.0,189.0
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.169,0.103,0.769,0.027,0.106,0.186,30.0,123.0,9.0,68.0
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.167,0.102,0.801,0.027,0.100,0.174,0.0,67.0,26.0,222.0
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.132,0.084,0.786,0.052,0.104,0.155,6.0,102.0,45.0,57.0
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.161,0.102,0.757,0.032,0.147,0.189,0.0,64.0,49.0,0.0
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.149,0.096,0.823,0.036,0.066,0.129,0.0,84.0,30.0,199.0
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.148,0.092,0.781,0.043,0.087,0.145,0.0,82.0,34.0,0.0
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.182,0.105,0.748,0.042,0.120,0.216,7.0,107.0,39.0,27.0
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.180,0.096,0.773,0.065,0.118,0.200,NaN,NaN,NaN,NaN


In [90]:
# S1901 and S1301 Imputation for numerical columns
numerical_columns_to_impute = ['# of Married Couple Families', 'Birth Rate Per 1000 Women (15-19 Years Old)',
                               'Birth Rate Per 1000 Women (20-34 Years Old)', 'Birth Rate Per 1000 Women (35-50 Years Old)',
                               'Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)']

numerical_imputer = SimpleImputer(strategy='mean')
merged_dfs_2023[numerical_columns_to_impute] = numerical_imputer.fit_transform(merged_dfs_2023[numerical_columns_to_impute])

# Display
merged_dfs_2023


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.162,0.096,0.821,0.035,0.079,0.141,0.000000,94.000000,49.000000,97.555556
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.146,0.090,0.811,0.039,0.110,0.144,9.000000,79.000000,35.000000,189.000000
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.169,0.103,0.769,0.027,0.106,0.186,30.000000,123.000000,9.000000,68.000000
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.167,0.102,0.801,0.027,0.100,0.174,0.000000,67.000000,26.000000,222.000000
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.132,0.084,0.786,0.052,0.104,0.155,6.000000,102.000000,45.000000,57.000000
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.161,0.102,0.757,0.032,0.147,0.189,0.000000,64.000000,49.000000,0.000000
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.149,0.096,0.823,0.036,0.066,0.129,0.000000,84.000000,30.000000,199.000000
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.148,0.092,0.781,0.043,0.087,0.145,0.000000,82.000000,34.000000,0.000000
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.182,0.105,0.748,0.042,0.120,0.216,7.000000,107.000000,39.000000,27.000000
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.180,0.096,0.773,0.065,0.118,0.200,6.589744,89.384615,32.102564,97.555556


In [91]:
# Dropping the underepresented 'Mean Married Couple Families Income ($)' columns with only (77) non-null values
final_2023_county_attributes = merged_dfs_2023.drop(columns='Mean Married Couple Families Income ($)')
final_2023_county_attributes


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.162,0.096,0.821,0.035,0.079,0.141,0.000000,94.000000,49.000000,97.555556
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.146,0.090,0.811,0.039,0.110,0.144,9.000000,79.000000,35.000000,189.000000
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.169,0.103,0.769,0.027,0.106,0.186,30.000000,123.000000,9.000000,68.000000
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.167,0.102,0.801,0.027,0.100,0.174,0.000000,67.000000,26.000000,222.000000
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.132,0.084,0.786,0.052,0.104,0.155,6.000000,102.000000,45.000000,57.000000
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.161,0.102,0.757,0.032,0.147,0.189,0.000000,64.000000,49.000000,0.000000
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.149,0.096,0.823,0.036,0.066,0.129,0.000000,84.000000,30.000000,199.000000
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.148,0.092,0.781,0.043,0.087,0.145,0.000000,82.000000,34.000000,0.000000
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.182,0.105,0.748,0.042,0.120,0.216,7.000000,107.000000,39.000000,27.000000
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.180,0.096,0.773,0.065,0.118,0.200,6.589744,89.384615,32.102564,97.555556


In [92]:
# Finally, confirm impuation process via value counts (all columns should = 120)
final_2023_county_attributes.count()


Year                                                                      40
County                                                                    40
% Male                                                                    40
% Female                                                                  40
% Male > 18 Years Old                                                     40
                                                                          ..
% Nonveteran Population With Any Disability                               40
Birth Rate Per 1000 Women (15-19 Years Old)                               40
Birth Rate Per 1000 Women (20-34 Years Old)                               40
Birth Rate Per 1000 Women (35-50 Years Old)                               40
Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)    40
Length: 102, dtype: int64

In [93]:
# Finally, confirm impuation process via value counts (all columns should = 120)
final_2023_county_attributes.count().tolist()


[40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40]

In [94]:
# Exporting dataset to CSV
final_2023_county_attributes.to_csv('Resources/2023_County_Final.csv', index=False)


**2023 County Dataset Unsupervised Learning Segmentation & Dataset Enrichment**
-
-----------

In [95]:
# Scaling the final 2023 county attribute table (which already does not include target/outcome variable (Winning Party - Democrat: 0 Republican: 1))
# Also must remove the 'County' column as it represents string categorical values
final_2023_county_attributes_no_county = final_2023_county_attributes.drop(columns=['County'])
final_2023_county_attributes_no_county


,Year,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,% High School Graduate (18-24),...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,2023,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,0.398,...,0.162,0.096,0.821,0.035,0.079,0.141,0.000000,94.000000,49.000000,97.555556
1,2023,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,0.374,...,0.146,0.090,0.811,0.039,0.110,0.144,9.000000,79.000000,35.000000,189.000000
2,2023,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,0.471,...,0.169,0.103,0.769,0.027,0.106,0.186,30.000000,123.000000,9.000000,68.000000
3,2023,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,0.431,...,0.167,0.102,0.801,0.027,0.100,0.174,0.000000,67.000000,26.000000,222.000000
4,2023,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,0.473,...,0.132,0.084,0.786,0.052,0.104,0.155,6.000000,102.000000,45.000000,57.000000
5,2023,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,0.387,...,0.161,0.102,0.757,0.032,0.147,0.189,0.000000,64.000000,49.000000,0.000000
6,2023,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,0.395,...,0.149,0.096,0.823,0.036,0.066,0.129,0.000000,84.000000,30.000000,199.000000
7,2023,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,0.487,...,0.148,0.092,0.781,0.043,0.087,0.145,0.000000,82.000000,34.000000,0.000000
8,2023,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,0.359,...,0.182,0.105,0.748,0.042,0.120,0.216,7.000000,107.000000,39.000000,27.000000
9,2023,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,0.541,...,0.180,0.096,0.773,0.065,0.118,0.200,6.589744,89.384615,32.102564,97.555556


In [96]:
# Scaling the 2023 final county attribute table WITHOUT the target/outcome variable (Winning Party - Democrat: 0 Republican: 1)
scaler = StandardScaler()
scaled_2023_final_county_attributes_no_county = scaler.fit_transform(final_2023_county_attributes_no_county)
scaled_2023_final_county_attributes_no_county


array([[ 0.00000000e+00, -5.46421554e-01,  5.46421554e-01, ...,
         2.28851827e-01,  1.21981769e+00, -1.40160727e-16],
       [ 0.00000000e+00, -9.56879060e-01,  9.56879060e-01, ...,
        -5.14916611e-01,  2.09164490e-01,  9.01910551e-01],
       [ 0.00000000e+00,  1.71879080e-01, -1.71879080e-01, ...,
         1.66680414e+00, -1.66776288e+00, -2.91504504e-01],
       ...,
       [ 0.00000000e+00, -4.43807178e-01,  4.43807178e-01, ...,
        -7.62839424e-01,  2.08609186e+00, -1.53423423e-02],
       [ 0.00000000e+00, -3.41192801e-01,  3.41192801e-01, ...,
         9.23035703e-01,  2.09164490e-01, -8.43828827e-02],
       [ 0.00000000e+00,  6.92647040e-02, -6.92647040e-02, ...,
         8.00981395e-02, -7.29299194e-01, -9.62184039e-01]])

In [97]:
# Creating a DataFrame with the scaled data
scaled_2023_final_county_attributes_no_county_df = pd.DataFrame(scaled_2023_final_county_attributes_no_county,
                                                                   columns=final_2023_county_attributes_no_county.columns)
scaled_2023_final_county_attributes_no_county_df


,Year,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,% High School Graduate (18-24),...,% Nonveteran Population 65-74 Years Old,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months)
0,0.0,-0.546422,0.546422,-0.260952,0.260952,0.531015,-0.643994,0.563235,-0.668954,-0.215248,...,0.737031,0.149986,1.083687,-0.522802,-0.962486,-0.955746,-0.515460,0.228852,1.219818,-1.401607e-16
1,0.0,-0.956879,0.956879,-0.928990,0.928990,-0.482404,0.945589,-0.430709,0.761513,-0.494111,...,-0.217827,-0.430605,0.849250,-0.193478,-0.082337,-0.851198,0.188534,-0.514917,0.209164,9.019106e-01
2,0.0,0.171879,-0.171879,0.323581,-0.323581,1.099518,-0.743343,-0.762024,-0.963461,0.632962,...,1.154782,0.827343,-0.135388,-1.181450,-0.195904,0.612479,1.831186,1.666804,-1.667763,-2.915045e-01
3,0.0,-0.238578,0.238578,-0.511467,0.511467,0.341514,-0.062093,-0.099394,-0.879316,0.168190,...,1.035424,0.730578,0.614812,-1.181450,-0.366256,0.194286,-0.515460,-1.109931,-0.440541,1.227387e+00
4,0.0,-0.238578,0.238578,-0.260952,0.260952,-1.034429,-0.246598,1.557179,-0.416518,0.656200,...,-1.053328,-1.011197,0.263156,0.876825,-0.252688,-0.467854,-0.046131,0.625528,0.931060,-3.999968e-01
5,0.0,-0.033350,0.033350,-0.093943,0.093943,0.720516,-0.487874,-0.762024,-0.668954,-0.343060,...,0.677352,0.730578,-0.416713,-0.769795,0.968164,0.717028,-0.515460,-1.258685,1.219818,-9.621840e-01
6,0.0,-0.238578,0.238578,-0.260952,0.260952,-0.070445,-0.374333,-0.099394,1.098094,-0.250106,...,-0.038791,0.149986,1.130575,-0.440471,-1.331581,-1.373940,-0.515460,-0.266994,-0.151783,1.000540e+00
7,0.0,0.069265,-0.069265,0.073067,-0.073067,0.910017,-0.700765,0.231920,-0.416518,0.818871,...,-0.098470,-0.237075,0.145937,0.135846,-0.735350,-0.816349,-0.515460,-0.366163,0.136975,-9.621840e-01
8,0.0,0.069265,-0.069265,0.073067,-0.073067,0.728755,-0.374333,-0.762024,-0.837244,-0.668401,...,1.930604,1.020873,-0.627706,0.053515,0.201583,1.657963,0.032091,0.873451,0.497923,-6.958848e-01
9,0.0,1.300637,-1.300637,1.242133,-1.242133,0.448623,-0.502067,4.539012,-0.711026,1.446314,...,1.811246,0.149986,-0.041613,1.947129,0.144799,1.100372,0.000000,0.000000,0.000000,-1.401607e-16


In [98]:
# Principal Component Analysis for dimensionality reduction of scaled data (30 components results in ~>95% explained variance ratio)
pca = PCA(n_components=30, random_state=0)
scaled_2023_pca_data = pca.fit_transform(scaled_2023_final_county_attributes_no_county_df)
print(f'Explained Variance Ratio: {sum(pca.explained_variance_ratio_)}')


Explained Variance Ratio: 0.9896285899339682


In [99]:
# Creating a new DataFrame with PCA data
scaled_2023_pca_df = pd.DataFrame(scaled_2023_pca_data, columns=["PCA1", "PCA2", "PCA3", "PCA4", "PCA5", "PCA6", "PCA7",
                                                       "PCA8", "PCA9", "PCA10", "PCA11", "PCA12", "PCA13", "PCA14", 
                                                       "PCA15", "PCA16", "PCA17", "PCA18", "PCA19", "PCA20", "PCA21", 
                                                       "PCA22", "PCA23", "PCA24", "PCA25", "PCA26", "PCA27", "PCA28", 
                                                       "PCA29", "PCA30"])
scaled_2023_pca_df


,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30
0,1.098080,-3.904122,-1.078552,2.297185,-1.520520,-1.251986,-1.792928,0.317405,-1.045325,-1.676282,...,0.603329,1.501338,-0.170263,1.407720,0.038791,-0.859187,-0.249214,-0.412694,-0.046805,-0.632698
1,7.061641,3.263824,0.019928,-2.276041,-2.004276,-0.003493,-1.018050,-2.504878,-0.755840,2.302556,...,-0.936606,-0.562003,-0.609230,-0.414293,0.630469,0.850881,-1.004101,-0.174910,0.069422,0.259953
2,-5.244941,-3.092681,-0.133826,-3.361732,-0.442613,0.969840,0.715150,-0.536695,0.265444,1.762124,...,2.103651,0.960414,-0.047877,-1.303450,0.127141,-0.189703,1.788178,-0.175934,-0.658580,0.116742
3,-0.549429,-3.013759,0.096991,-1.091579,-3.303276,0.101632,-2.170505,2.520063,-0.751517,0.755642,...,-0.759090,0.566222,1.041460,0.390007,-0.354378,-0.242139,-0.983924,-0.107164,0.370336,-0.160868
4,1.781413,0.823923,-1.988176,1.657032,2.443888,-2.036446,0.506084,-0.005241,2.132839,-1.467891,...,-0.108231,-0.398222,-0.234586,0.020503,-1.150037,0.109887,0.323037,0.428277,0.446900,0.110688
5,-6.850087,0.421004,1.126390,0.024895,-2.000069,0.036540,-2.217604,-0.517604,-1.544651,0.595670,...,-0.328913,-0.957652,-0.076994,-0.269233,-0.059025,-0.417588,-0.097835,0.164465,-0.528006,-0.714784
6,11.954336,-3.107610,0.563458,-3.762317,0.434458,1.003146,-0.361021,-0.041546,-1.326247,-1.253665,...,-0.845338,-0.847824,-0.550289,-0.062468,-0.535144,-0.253055,1.120022,0.373645,0.701622,-0.583581
7,4.457168,-2.158398,1.563689,-1.332378,0.393640,-0.267568,1.352884,0.698624,-0.003330,-0.487920,...,-0.753113,-0.689575,0.956885,1.256771,0.324338,0.499523,-0.601189,-0.920403,-0.424829,0.483148
8,-6.302856,-0.985673,0.827041,-2.484405,-2.428857,-1.499697,0.270724,-0.967852,-0.682218,-1.183086,...,1.120265,0.748528,0.085992,1.679963,0.694964,2.225643,0.447432,0.337150,-0.270160,-0.685240
9,-4.466621,-3.117614,-2.353326,-0.185528,3.365957,-5.587601,0.458473,-1.936176,1.346078,0.339349,...,0.489803,-0.811259,0.041659,-0.716319,1.261794,-0.341538,0.012911,-0.230770,-0.400608,-0.029976


In [100]:
# Finding best value for K in Kmeans via elbow method
k = list(range(1, 11))
inertia = []
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(scaled_2023_pca_df)
    inertia.append(model.inertia_)

elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

df_elbow_data_pca = pd.DataFrame(elbow_data_pca)
df_elbow_data_pca


/Applications/anaconda3/envs/Eclipse/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Applications/anaconda3/envs/Eclipse/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Applications/anaconda3/envs/Eclipse/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Applications/anaconda3/envs/Eclipse/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureW

,k,inertia
0,1,3958.514360
1,2,2952.409794
2,3,2538.588107
3,4,2257.682978
4,5,2146.940683
5,6,1965.381063
6,7,1772.675724
7,8,1637.634889
8,9,1532.560733
9,10,1430.632399


In [101]:
# Visualize Elbow data (shows optimal value of k = 4)
elbow_plot_pca = df_elbow_data_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA(30) w/ Scaled Data", xticks=k)
elbow_plot_pca


:Curve   [k]   (inertia)

In [102]:
# Segment data with the determined number of clusters (k) = 4
model = KMeans(n_clusters=4, random_state=0)
model.fit(scaled_2023_pca_df)
segment_predictions = model.predict(scaled_2023_pca_df)
print(segment_predictions)


/Applications/anaconda3/envs/Eclipse/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[2 0 1 2 2 1 0 2 1 1 1 0 1 2 1 2 2 0 1 1 2 1 1 2 1 2 2 1 1 1 2 0 2 1 3 1 1
 2 2 2]


In [103]:
# Enrich master 2023 county dataset that has target/label outcomes included with segment predictions
ultimate_2023_county_df = final_2023_county_attributes.copy()
ultimate_2023_county_df['Segment'] = segment_predictions
ultimate_2023_county_df


,Year,County,% Male,% Female,% Male > 18 Years Old,% Female > 18 Years Old,% White,% Black or African American,% American Indian and Alaska Native,% Asian,...,% Nonveteran Population 75 Years Old & Over,Nonveteran Labor Force Participation Rate (%),Nonveteran Unemployment Rate (%),% Nonveteran Population With Income Below Poverty Level (Past 12 Months),% Nonveteran Population With Any Disability,Birth Rate Per 1000 Women (15-19 Years Old),Birth Rate Per 1000 Women (20-34 Years Old),Birth Rate Per 1000 Women (35-50 Years Old),Received Public Assistance Income Rate Per 1000 Women (Past 12 Months),Segment
0,2023,ADAMS,0.492,0.508,0.490,0.510,0.882,0.013,0.004,0.008,...,0.096,0.821,0.035,0.079,0.141,0.000000,94.000000,49.000000,97.555556,2
1,2023,ALLEGHENY,0.488,0.512,0.482,0.518,0.759,0.125,0.001,0.042,...,0.090,0.811,0.039,0.110,0.144,9.000000,79.000000,35.000000,189.000000,0
2,2023,ARMSTRONG,0.499,0.501,0.497,0.503,0.951,0.006,0.000,0.001,...,0.103,0.769,0.027,0.106,0.186,30.000000,123.000000,9.000000,68.000000,1
3,2023,BEAVER,0.495,0.505,0.487,0.513,0.859,0.054,0.002,0.003,...,0.102,0.801,0.027,0.100,0.174,0.000000,67.000000,26.000000,222.000000,2
4,2023,BERKS,0.495,0.505,0.490,0.510,0.692,0.041,0.007,0.014,...,0.084,0.786,0.052,0.104,0.155,6.000000,102.000000,45.000000,57.000000,2
5,2023,BLAIR,0.497,0.503,0.492,0.508,0.905,0.024,0.000,0.008,...,0.102,0.757,0.032,0.147,0.189,0.000000,64.000000,49.000000,0.000000,1
6,2023,BUCKS,0.495,0.505,0.490,0.510,0.809,0.032,0.002,0.050,...,0.096,0.823,0.036,0.066,0.129,0.000000,84.000000,30.000000,199.000000,0
7,2023,BUTLER,0.498,0.502,0.494,0.506,0.928,0.009,0.003,0.014,...,0.092,0.781,0.043,0.087,0.145,0.000000,82.000000,34.000000,0.000000,2
8,2023,CAMBRIA,0.498,0.502,0.494,0.506,0.906,0.032,0.000,0.004,...,0.105,0.748,0.042,0.120,0.216,7.000000,107.000000,39.000000,27.000000,1
9,2023,CARBON,0.510,0.490,0.508,0.492,0.872,0.023,0.016,0.007,...,0.096,0.773,0.065,0.118,0.200,6.589744,89.384615,32.102564,97.555556,1


In [104]:
# Export 2023 final county dataset with segmentation column included for model extrapolation (executed in another notebook)
ultimate_2023_county_df.to_csv('Resources/2023_County_Final_With_Segments.csv', index=False)
